In [1]:
from datetime import datetime
import datetime as dt
import time
from subgrounds.subgraph import SyntheticField, FieldPath
from subgrounds.subgrounds import Subgrounds
import pandas as pd
import duckdb as db
import IPython as ip
import  os as os

In [2]:
sg = Subgrounds()
apiKey = input('API Key, please')

In [3]:
#for when we save the files
file = input('Selet a folder to save output') ##enter your file path here - the file is in the repo "summary_stats.csv".

*******************************************************************
*******************************************************************
LET'S LOOK AT SILO FIRST

In [95]:
#SILO subgraph: https://gateway.thegraph.com/api/[api-key]/subgraphs/id/6rcT2GzBXGoi3ZMus92LHajM2b9qWGj7Z3dDhPxoUogB
silo = sg.load_api('https://gateway.thegraph.com/api/'+apiKey+'/subgraphs/id/6rcT2GzBXGoi3ZMus92LHajM2b9qWGj7Z3dDhPxoUogB')

##this captures the list of people who borrowed XAI against gOHM
borrows = silo.Query.borrows(
    orderBy='timestamp',
    orderDirection='desc',
    first=1000,
    #skip = skipValue,
    where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' }

)

borrowsDb = sg.query_df([
    borrows.id
    ,borrows.hash
    ,borrows.logIndex
    ,borrows.blockNumber
    ,borrows.timestamp
    ,borrows.account
    ,borrows.market
    ,borrows.market.rates
    ,borrows.asset
    ,borrows.amount
    ,borrows.amountUSD 
])
borrowsDb.head(100)

#this, in turn, pulls repayment events
repays = silo.Query.repays(
    orderBy='timestamp',
    orderDirection='desc',
    first=1000,
    #skip = skipValue,
    where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' }

)

repaysDb = sg.query_df([
    repays.id
    ,repays.hash
    ,repays.logIndex
    ,repays.blockNumber
    ,repays.timestamp
    ,repays.account
    ,repays.market
    ,repays.market.rates
    ,repays.asset
    ,repays.amount
    ,repays.amountUSD
    ,repays.position
])
repaysDb.head(100)

In [96]:
##this captures the ENTIRE list of people who borrowed XAI against gOHM
borrowTicker = 0
borrowslist = pd.DataFrame()
borrowsDb = pd.DataFrame()
borrowListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (borrowTicker)*(1000)
    borrows = silo.Query.borrows(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' } #gOHM market

    )

    borrowslist = sg.query_df([
        borrows.id
        ,borrows.hash
        ,borrows.logIndex
        ,borrows.blockNumber
        ,borrows.timestamp
        ,borrows.account
        ,borrows.market
        ,borrows.market.rates
        ,borrows.asset
        ,borrows.amount
        ,borrows.amountUSD
        ,borrows.position
    ])

    #borrowslist.sort_values(by=['borrows_timestamp'])

    borrowsDb=pd.concat([borrowsDb, borrowslist])
    recordTimestamp1 = borrowsDb.iat[borrowTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    borrowsDbLength = len(borrowsDb)
    borrowListLength = len(borrowslist)
    recordID = borrowsDb.iat[borrowTicker,0]
    borrow = borrowsDb.iat[borrowTicker,1]
    print("iterations: ", borrowTicker, "Lines skipped: ",skipValue, "records collected: ", borrowListLength, " - latest recordID: ",borrow, " ", recordID , " - borrows DB length: ", borrowsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", borrowTicker, "records collected: ", borrowListLength, " - userdb length: ", borrowsDbLength)
    borrowTicker = borrowTicker+1

    if daysAgo>daysLimit: exit =True
    if borrowListLength<1000: exit =True

borrowsDb.columns = borrowsDb.columns.str.replace("borrows", "transaction")
borrowsDb['event']='borrow'
print("Done. Total Records collected: ", borrowsDbLength)

iterations:  0 Lines skipped:  0 records collected:  114  - latest recordID:  0x97811f7e7b7d202797093d44047ed986054be3dd6120bce6c03d3172ab5525b7   0x97811f7e7b7d202797093d44047ed986054be3dd6120bce6c03d3172ab5525b7-23  - borrows DB length:  114  - latest record from:  2023-02-13 15:17:35 0 days ago
Done. Total Records collected:  114


In [97]:
#I swear I will figure out synthetic fields, but until then...
borrowsDbTimed = db.query("select "
               "    * "
                    " ,to_timestamp(transaction_timestamp::int) dateTime "
               "From borrowsDb "
               #"where lower(liquidityPools_name) like lower('%olympus%') "
                #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
               #" limit 100 "
               ).df()
borrowsDbTimed.head(100)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event,dateTime
0,0x97811f7e7b7d202797093d44047ed986054be3dd6120...,0x97811f7e7b7d202797093d44047ed986054be3dd6120...,23,16623063,1676330255,0xc0b1f200aa9546ce074397bed7438bb2ec033877,3,2,1,4,...,BORROWER,True,5547516074488804244751,0,0,2,0,0,borrow,2023-02-13 23:17:35
1,0x97811f7e7b7d202797093d44047ed986054be3dd6120...,0x97811f7e7b7d202797093d44047ed986054be3dd6120...,23,16623063,1676330255,0xc0b1f200aa9546ce074397bed7438bb2ec033877,3,2,1,4,...,BORROWER,True,5547516074488804244751,0,0,2,0,0,borrow,2023-02-13 23:17:35
2,0x97811f7e7b7d202797093d44047ed986054be3dd6120...,0x97811f7e7b7d202797093d44047ed986054be3dd6120...,23,16623063,1676330255,0xc0b1f200aa9546ce074397bed7438bb2ec033877,3,2,1,4,...,BORROWER,True,5547516074488804244751,0,0,2,0,0,borrow,2023-02-13 23:17:35
3,0x97811f7e7b7d202797093d44047ed986054be3dd6120...,0x97811f7e7b7d202797093d44047ed986054be3dd6120...,23,16623063,1676330255,0xc0b1f200aa9546ce074397bed7438bb2ec033877,3,2,1,4,...,BORROWER,True,5547516074488804244751,0,0,2,0,0,borrow,2023-02-13 23:17:35
4,0x97811f7e7b7d202797093d44047ed986054be3dd6120...,0x97811f7e7b7d202797093d44047ed986054be3dd6120...,23,16623063,1676330255,0xc0b1f200aa9546ce074397bed7438bb2ec033877,3,2,1,4,...,BORROWER,True,5547516074488804244751,0,0,2,0,0,borrow,2023-02-13 23:17:35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0x2485acd5a3853dc50921a27ea5bf8227871425cca3e4...,0x2485acd5a3853dc50921a27ea5bf8227871425cca3e4...,147,16171818,1670886647,0x1ba67f30e25fc43863b50047d7fd4bd63b9c59ec,7,5,2,10,...,BORROWER,True,11900000000000000000000,0,0,1,0,0,borrow,2022-12-12 23:10:47
96,0x17fedebf20b120a55b895f96e2b466f2bbdcceb2b542...,0x17fedebf20b120a55b895f96e2b466f2bbdcceb2b542...,113,16171351,1670881019,0x3985caf03eef8f885edb1f8358143bdf08d8d48b,2,2,0,1,...,BORROWER,True,4000000000000000000000,0,0,1,0,0,borrow,2022-12-12 21:36:59
97,0x17fedebf20b120a55b895f96e2b466f2bbdcceb2b542...,0x17fedebf20b120a55b895f96e2b466f2bbdcceb2b542...,113,16171351,1670881019,0x3985caf03eef8f885edb1f8358143bdf08d8d48b,2,2,0,1,...,BORROWER,True,4000000000000000000000,0,0,1,0,0,borrow,2022-12-12 21:36:59
98,0x17fedebf20b120a55b895f96e2b466f2bbdcceb2b542...,0x17fedebf20b120a55b895f96e2b466f2bbdcceb2b542...,113,16171351,1670881019,0x3985caf03eef8f885edb1f8358143bdf08d8d48b,2,2,0,1,...,BORROWER,True,4000000000000000000000,0,0,1,0,0,borrow,2022-12-12 21:36:59


In [98]:
path =file+'/siloFinance_borrows_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(len(borrowsDbTimed))+'_records.csv'
borrowsDbTimed.to_csv(path, index = False)

path =file+'/siloFinance_borrows_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(len(borrowsDbTimed))+'_records_piped.csv'
borrowsDbTimed.to_csv(path, index = False, sep='|')
print('file saved')

file saved


In [99]:
##this tracks the ENTIRE list of events for repaying the XAI repayed against gOHM
repayTicker = 0
repayslist = pd.DataFrame()
repaysDb = pd.DataFrame()
repayListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (repayTicker)*(1000)
    repays = silo.Query.repays(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' } #gOHM market

    )

    repayslist = sg.query_df([
        repays.id
        ,repays.hash
        ,repays.logIndex
        ,repays.blockNumber
        ,repays.timestamp
        ,repays.account
        ,repays.market
        ,repays.market.rates
        ,repays.asset
        ,repays.amount
        ,repays.amountUSD
        ,repays.position
    ])

    #repayslist.sort_values(by=['repays_timestamp'])

    repaysDb=pd.concat([repaysDb, repayslist])
    recordTimestamp1 = repaysDb.iat[repayTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    repaysDbLength = len(repaysDb)
    repayListLength = len(repayslist)
    recordID = repaysDb.iat[repayTicker,0]
    repay = repaysDb.iat[repayTicker,1]
    print("iterations: ", repayTicker, "Lines skipped: ",skipValue, "records collected: ", repayListLength, " - latest recordID: ",repay, " ", recordID , " - repays DB length: ", repaysDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", repayTicker, "records collected: ", repayListLength, " - userdb length: ", repaysDbLength)
    repayTicker = repayTicker+1

    if daysAgo>daysLimit: exit =True
    if repayListLength<1000: exit =True

repaysDb.columns = repaysDb.columns.str.replace("repays", "transaction")
repaysDb['event']='repay'
print("Done. Total Records collected: ", repaysDbLength)

iterations:  0 Lines skipped:  0 records collected:  12  - latest recordID:  0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad01a68715d0385e8b26cd   0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad01a68715d0385e8b26cd-242  - repays DB length:  12  - latest record from:  2022-11-25 13:03:47 80 days ago
Done. Total Records collected:  12


In [100]:
repaysDb.head(10)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_timestampClosed,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event
0,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,242,16049560,1669410227,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,1669410227,BORROWER,True,0,0,0,1,1,0,repay
1,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,242,16049560,1669410227,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,1669410227,BORROWER,True,0,0,0,1,1,0,repay
2,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,242,16049560,1669410227,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,1669410227,BORROWER,True,0,0,0,1,1,0,repay
3,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,242,16049560,1669410227,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,1669410227,BORROWER,True,0,0,0,1,1,0,repay
4,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,242,16049560,1669410227,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,1669410227,BORROWER,True,0,0,0,1,1,0,repay
5,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,242,16049560,1669410227,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,1669410227,BORROWER,True,0,0,0,1,1,0,repay
6,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,200,16036129,1669248167,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,2,0,2,2,...,1669248167,BORROWER,True,0,0,0,1,1,0,repay
7,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,200,16036129,1669248167,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,2,0,2,2,...,1669248167,BORROWER,True,0,0,0,1,1,0,repay
8,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,200,16036129,1669248167,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,2,0,2,2,...,1669248167,BORROWER,True,0,0,0,1,1,0,repay
9,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,200,16036129,1669248167,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,2,0,2,2,...,1669248167,BORROWER,True,0,0,0,1,1,0,repay


In [101]:
#this tracks the ENTIRE list of events for depositing gOHM for lending
depositTicker = 0
depositslist = pd.DataFrame()
depositsDb = pd.DataFrame()
depositListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (depositTicker)*(1000)
    deposits = silo.Query.deposits(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' } #gOHM market

    )

    depositslist = sg.query_df([
        deposits.id
        ,deposits.hash
        ,deposits.logIndex
        ,deposits.blockNumber
        ,deposits.timestamp
        ,deposits.account
        ,deposits.market
        ,deposits.market.rates
        ,deposits.asset
        ,deposits.amount
        ,deposits.amountUSD
        ,deposits.position
    ])

    #depositslist.sort_values(by=['deposits_timestamp'])

    depositsDb=pd.concat([depositsDb, depositslist])
    recordTimestamp1 = depositsDb.iat[depositTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    depositsDbLength = len(depositsDb)
    depositListLength = len(depositslist)
    recordID = depositsDb.iat[depositTicker,0]
    deposit = depositsDb.iat[depositTicker,1]
    print("iterations: ", depositTicker, "Lines skipped: ",skipValue, "records collected: ", depositListLength, " - latest recordID: ",deposit, " ", recordID , " - deposits DB length: ", depositsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", depositTicker, "records collected: ", depositListLength, " - userdb length: ", depositsDbLength)
    depositTicker = depositTicker+1
    if daysAgo>daysLimit: exit =True
    if depositListLength<1000: exit =True

#depositsDb.sort_values(by=['deposits_timestamp'])

depositsDb.columns = depositsDb.columns.str.replace("deposits", "transaction")
depositsDb['event']='deposit'
print("Done. Total Records collected: ", depositsDbLength)




iterations:  0 Lines skipped:  0 records collected:  252  - latest recordID:  0xf5439a3f94551476ec68db02d6f577a2856d01f45691189ad831ad8cb0f5019a   0xf5439a3f94551476ec68db02d6f577a2856d01f45691189ad831ad8cb0f5019a-246  - deposits DB length:  252  - latest record from:  2023-02-13 15:18:59 0 days ago
Done. Total Records collected:  252


In [102]:
#I swear I will figure out synthetic fields, but until then...
depositsDbTimed = db.query("select "
               "    * "
                    " ,to_timestamp(transaction_timestamp::int) dateTime "
               "From depositsDb "
               #"where lower(liquidityPools_name) like lower('%olympus%') "
                #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
               #" limit 100 "
               ).df()
depositsDbTimed.head(100)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event,dateTime
0,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,246,16623070,1676330339,0xc0b1f200aa9546ce074397bed7438bb2ec033877,3,2,1,4,...,LENDER,True,3792621600516793563,3,0,0,0,0,deposit,2023-02-13 23:18:59
1,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,246,16623070,1676330339,0xc0b1f200aa9546ce074397bed7438bb2ec033877,3,2,1,4,...,LENDER,True,3792621600516793563,3,0,0,0,0,deposit,2023-02-13 23:18:59
2,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,246,16623070,1676330339,0xc0b1f200aa9546ce074397bed7438bb2ec033877,3,2,1,4,...,LENDER,True,3792621600516793563,3,0,0,0,0,deposit,2023-02-13 23:18:59
3,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,246,16623070,1676330339,0xc0b1f200aa9546ce074397bed7438bb2ec033877,3,2,1,4,...,LENDER,True,3792621600516793563,3,0,0,0,0,deposit,2023-02-13 23:18:59
4,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,246,16623070,1676330339,0xc0b1f200aa9546ce074397bed7438bb2ec033877,3,2,1,4,...,LENDER,True,3792621600516793563,3,0,0,0,0,deposit,2023-02-13 23:18:59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0x4d72e2a5ca35d7b76faecafddc8dce2368081f6c6a52...,0x4d72e2a5ca35d7b76faecafddc8dce2368081f6c6a52...,256,16574706,1675746299,0x6487d336bdec94d4714a4d217b15aa5c7b9dfa99,2,2,0,1,...,LENDER,True,6076244450105671195,1,0,0,0,0,deposit,2023-02-07 05:04:59
96,0x2051da675e864d363e7159d1e0084e45535e785b5ae6...,0x2051da675e864d363e7159d1e0084e45535e785b5ae6...,230,16573196,1675728083,0x36125234dd6c7d0a01dcadc67e79e061d405098e,2,2,0,1,...,LENDER,True,14004860882719985729,1,0,0,0,0,deposit,2023-02-07 00:01:23
97,0x2051da675e864d363e7159d1e0084e45535e785b5ae6...,0x2051da675e864d363e7159d1e0084e45535e785b5ae6...,230,16573196,1675728083,0x36125234dd6c7d0a01dcadc67e79e061d405098e,2,2,0,1,...,LENDER,True,14004860882719985729,1,0,0,0,0,deposit,2023-02-07 00:01:23
98,0x2051da675e864d363e7159d1e0084e45535e785b5ae6...,0x2051da675e864d363e7159d1e0084e45535e785b5ae6...,230,16573196,1675728083,0x36125234dd6c7d0a01dcadc67e79e061d405098e,2,2,0,1,...,LENDER,True,14004860882719985729,1,0,0,0,0,deposit,2023-02-07 00:01:23


In [103]:
path =file+'/siloFinance_deposits_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(len(depositsDbTimed))+'_records.csv'
depositsDbTimed.to_csv(path, index = False)

path =file+'/siloFinance_deposits_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(len(depositsDbTimed))+'_records_piped.csv'
depositsDbTimed.to_csv(path, index = False, sep='|')
print('file saved')

file saved


In [104]:
print("depoosit rows ",len(depositsDb) ,", borrow rows ", len(borrowsDb),", repay rows ", len(repaysDb), ". Expected rows: ",(len(depositsDb)+len(borrowsDb))+len(repaysDb) )

depoosit rows  252 , borrow rows  114 , repay rows  12 . Expected rows:  378


In [105]:
siloEventsDb = pd.DataFrame()
siloEventsDb=pd.concat([depositsDb, borrowsDb,repaysDb])
siloEventsDb.head(10)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_timestampClosed,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event
0,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,246,16623070,1676330339,0xc0b1f200aa9546ce074397bed7438bb2ec033877,3,2,1,4,...,NaN,LENDER,True,3792621600516793563,3,0,0,0,0,deposit
1,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,246,16623070,1676330339,0xc0b1f200aa9546ce074397bed7438bb2ec033877,3,2,1,4,...,NaN,LENDER,True,3792621600516793563,3,0,0,0,0,deposit
2,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,246,16623070,1676330339,0xc0b1f200aa9546ce074397bed7438bb2ec033877,3,2,1,4,...,NaN,LENDER,True,3792621600516793563,3,0,0,0,0,deposit
3,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,246,16623070,1676330339,0xc0b1f200aa9546ce074397bed7438bb2ec033877,3,2,1,4,...,NaN,LENDER,True,3792621600516793563,3,0,0,0,0,deposit
4,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,246,16623070,1676330339,0xc0b1f200aa9546ce074397bed7438bb2ec033877,3,2,1,4,...,NaN,LENDER,True,3792621600516793563,3,0,0,0,0,deposit
5,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,246,16623070,1676330339,0xc0b1f200aa9546ce074397bed7438bb2ec033877,3,2,1,4,...,NaN,LENDER,True,3792621600516793563,3,0,0,0,0,deposit
6,0x3069fa3e9834fd75b4eaae5d1ef1a7910b112ed68e5f...,0x3069fa3e9834fd75b4eaae5d1ef1a7910b112ed68e5f...,89,16623049,1676330087,0xc0b1f200aa9546ce074397bed7438bb2ec033877,3,2,1,4,...,NaN,LENDER,True,3792621600516793563,3,0,0,0,0,deposit
7,0x3069fa3e9834fd75b4eaae5d1ef1a7910b112ed68e5f...,0x3069fa3e9834fd75b4eaae5d1ef1a7910b112ed68e5f...,89,16623049,1676330087,0xc0b1f200aa9546ce074397bed7438bb2ec033877,3,2,1,4,...,NaN,LENDER,True,3792621600516793563,3,0,0,0,0,deposit
8,0x3069fa3e9834fd75b4eaae5d1ef1a7910b112ed68e5f...,0x3069fa3e9834fd75b4eaae5d1ef1a7910b112ed68e5f...,89,16623049,1676330087,0xc0b1f200aa9546ce074397bed7438bb2ec033877,3,2,1,4,...,NaN,LENDER,True,3792621600516793563,3,0,0,0,0,deposit
9,0x3069fa3e9834fd75b4eaae5d1ef1a7910b112ed68e5f...,0x3069fa3e9834fd75b4eaae5d1ef1a7910b112ed68e5f...,89,16623049,1676330087,0xc0b1f200aa9546ce074397bed7438bb2ec033877,3,2,1,4,...,NaN,LENDER,True,3792621600516793563,3,0,0,0,0,deposit


In [106]:
len(siloEventsDb)

378

In [107]:
#I swear I will figure out synthetic fields, but until then...
siloEventsDbTimed = db.query("select "
               "    * "
                    " ,to_timestamp(transaction_timestamp::int) dateTime "
               "From siloEventsDb "
               #"where lower(liquidityPools_name) like lower('%olympus%') "
                #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
               #" limit 100 "
               ).df()
siloEventsDbTimed.head(100)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event,dateTime
0,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,246,16623070,1676330339,0xc0b1f200aa9546ce074397bed7438bb2ec033877,3,2,1,4,...,LENDER,True,3792621600516793563,3,0,0,0,0,deposit,2023-02-13 23:18:59
1,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,246,16623070,1676330339,0xc0b1f200aa9546ce074397bed7438bb2ec033877,3,2,1,4,...,LENDER,True,3792621600516793563,3,0,0,0,0,deposit,2023-02-13 23:18:59
2,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,246,16623070,1676330339,0xc0b1f200aa9546ce074397bed7438bb2ec033877,3,2,1,4,...,LENDER,True,3792621600516793563,3,0,0,0,0,deposit,2023-02-13 23:18:59
3,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,246,16623070,1676330339,0xc0b1f200aa9546ce074397bed7438bb2ec033877,3,2,1,4,...,LENDER,True,3792621600516793563,3,0,0,0,0,deposit,2023-02-13 23:18:59
4,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,246,16623070,1676330339,0xc0b1f200aa9546ce074397bed7438bb2ec033877,3,2,1,4,...,LENDER,True,3792621600516793563,3,0,0,0,0,deposit,2023-02-13 23:18:59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0x4d72e2a5ca35d7b76faecafddc8dce2368081f6c6a52...,0x4d72e2a5ca35d7b76faecafddc8dce2368081f6c6a52...,256,16574706,1675746299,0x6487d336bdec94d4714a4d217b15aa5c7b9dfa99,2,2,0,1,...,LENDER,True,6076244450105671195,1,0,0,0,0,deposit,2023-02-07 05:04:59
96,0x2051da675e864d363e7159d1e0084e45535e785b5ae6...,0x2051da675e864d363e7159d1e0084e45535e785b5ae6...,230,16573196,1675728083,0x36125234dd6c7d0a01dcadc67e79e061d405098e,2,2,0,1,...,LENDER,True,14004860882719985729,1,0,0,0,0,deposit,2023-02-07 00:01:23
97,0x2051da675e864d363e7159d1e0084e45535e785b5ae6...,0x2051da675e864d363e7159d1e0084e45535e785b5ae6...,230,16573196,1675728083,0x36125234dd6c7d0a01dcadc67e79e061d405098e,2,2,0,1,...,LENDER,True,14004860882719985729,1,0,0,0,0,deposit,2023-02-07 00:01:23
98,0x2051da675e864d363e7159d1e0084e45535e785b5ae6...,0x2051da675e864d363e7159d1e0084e45535e785b5ae6...,230,16573196,1675728083,0x36125234dd6c7d0a01dcadc67e79e061d405098e,2,2,0,1,...,LENDER,True,14004860882719985729,1,0,0,0,0,deposit,2023-02-07 00:01:23


In [108]:
siloEventsDblength = len(siloEventsDbTimed)
siloEventsDblength

378

In [109]:
path =file+'/siloFinance_events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(siloEventsDblength)+'_records.csv'
siloEventsDbTimed.to_csv(path, index = False)

path =file+'/siloFinance_events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(siloEventsDblength)+'_records_piped.csv'
siloEventsDbTimed.to_csv(path, index = False, sep='|')
print('file saved')

file saved


In [110]:
#let's pull interest rate data
interestRateTicker = 0
interestRateslist = pd.DataFrame()
interestRatesDb = pd.DataFrame()
interestRateListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (interestRateTicker)*(1000)
    interestRates = silo.Query.interestRates(
        orderBy='id',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' } #gOHM market

    )

    interestRateslist = sg.query_df([
        interestRates.id
        ,interestRates.rate #5.21% should be stored as 5.21
        ,interestRates.duration #in days
        ,interestRates.maturityBlock #Maturity of the loan in block height. Only applies to fixed term lending (e.g. Notional)
        ,interestRates.side #The party the interest is paid to / received from
        ,interestRates.type #The type of interest rate (e.g. stable, fixed, variable, etc)
        ,interestRates.market
        ,interestRates.marketAsset
        ,interestRates.token
    ])

    #interestRateslist.sort_values(by=['interestRates_timestamp'])

    interestRatesDb=pd.concat([interestRateslist, interestRatesDb])
    interestRatesDbLength = len(interestRatesDb)
    interestRateListLength = len(interestRateslist)
    print("iterations: ", interestRateTicker, "Lines skipped: ",skipValue, "records collected: ", interestRateListLength)
    #print("iterations: ", interestRateTicker, "records collected: ", interestRateListLength, " - userdb length: ", interestRatesDbLength)
    interestRateTicker = interestRateTicker+1
    if interestRateListLength<1000: exit =True

print("Done. Total Records collected: ", interestRatesDbLength)




iterations:  0 Lines skipped:  0 records collected:  6
Done. Total Records collected:  6


In [111]:
interestRatesDb.head(10)

,interestRates_id,interestRates_rate,interestRates_duration,interestRates_maturityBlock,interestRates_side,interestRates_type,interestRates_market_id,interestRates_market_protocol_id,interestRates_market_name,interestRates_market_isActive,...,interestRates_token_symbol,interestRates_token_decimals,interestRates_token_type,interestRates_token_lastPriceUSD,interestRates_token_lastPriceBlockNumber,interestRates_token_market_id,interestRates_token_asset_id,interestRates_token_derivativeConversion,interestRates_token_totalSupply,interestRates_token_activeOracle_id
0,LENDER-VARIABLE-0xc413dd03555f3eb29d834b482d38...,2.225722,None,None,LENDER,VARIABLE,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,gOHM,True,...,XAI,18,Asset,1.001456,16624740,None,None,1.0,0,0xe37b8c83138caf12e57632d19c06eb561d47e423-0xd...
1,LENDER-VARIABLE-0xc413dd03555f3eb29d834b482d38...,0.000000,None,None,LENDER,VARIABLE,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,gOHM,True,...,WETH,18,Asset,1502.717438,16624740,None,None,1.0,0,None
2,LENDER-VARIABLE-0xc413dd03555f3eb29d834b482d38...,0.111839,None,None,LENDER,VARIABLE,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,gOHM,True,...,gOHM,18,Asset,2752.852892,16624740,None,None,1.0,0,0xba53d3a08fb8dd4309881b4e8a3bc386ea0ddb1f-0x0...
3,BORROWER-VARIABLE-0xc413dd03555f3eb29d834b482d...,3.335953,None,None,BORROWER,VARIABLE,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,gOHM,True,...,XAI,18,Asset,1.001456,16624740,None,None,1.0,0,0xe37b8c83138caf12e57632d19c06eb561d47e423-0xd...
4,BORROWER-VARIABLE-0xc413dd03555f3eb29d834b482d...,0.000000,None,None,BORROWER,VARIABLE,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,gOHM,True,...,WETH,18,Asset,1502.717438,16624740,None,None,1.0,0,None
5,BORROWER-VARIABLE-0xc413dd03555f3eb29d834b482d...,0.819168,None,None,BORROWER,VARIABLE,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,gOHM,True,...,gOHM,18,Asset,2752.852892,16624740,None,None,1.0,0,0xba53d3a08fb8dd4309881b4e8a3bc386ea0ddb1f-0x0...


In [112]:
interestRatesDbLength = len(interestRatesDb)
path =file+'/siloFinance_interestRates_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(interestRatesDbLength)+'_records.csv'
interestRatesDb.to_csv(path, index = False)

path =file+'/siloFinance_interestRates_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(interestRatesDbLength)+'_records_piped.csv'
interestRatesDb.to_csv(path, index = False, sep='|')
print('file saved')

file saved


********************************************************************
********************************************************************


LET'S LOOK AT VESTA WHILE WE'RE ON A RUN

Annoying: the schema for repay events is completely different from borrow and deposit. Can't union them into a single table without crunching it first. Argh.

Interest Rate data is weird, too. But I may not need it?

In [57]:
#https://api.thegraph.com/subgraphs/name/corerouter/vesta-finance
vesta = sg.load_api('https://api.thegraph.com/subgraphs/name/corerouter/vesta-finance')

In [70]:
##this captures the list of people who vestaBorrowed XAI against gOHM
vestaBorrowTicker = 0
vestaBorrowslist = pd.DataFrame()
vestaBorrowsDb = pd.DataFrame()
vestaBorrowListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (vestaBorrowTicker)*(1000)
    vestaBorrows = vesta.Query.borrows(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1' } #gOHM market

    )

    vestaBorrowslist = sg.query_df([
        vestaBorrows.id
        ,vestaBorrows.hash
        ,vestaBorrows.logIndex
        ,vestaBorrows.blockNumber
        ,vestaBorrows.timestamp
        ,vestaBorrows.account
        ,vestaBorrows.market
        ,vestaBorrows.market.rates
        ,vestaBorrows.asset
        ,vestaBorrows.amount
        ,vestaBorrows.amountUSD
        ,vestaBorrows.position
    ])

    #vestaBorrowslist.sort_values(by=['vestaBorrows_timestamp'])

    vestaBorrowsDb=pd.concat([vestaBorrowsDb, vestaBorrowslist])
    recordTimestamp1 = vestaBorrowsDb.iat[vestaBorrowTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    vestaBorrowsDbLength = len(vestaBorrowsDb)
    vestaBorrowListLength = len(vestaBorrowslist)
    recordID = vestaBorrowsDb.iat[vestaBorrowTicker,0]
    vestaBorrow = vestaBorrowsDb.iat[vestaBorrowTicker,1]
    print("iterations: ", vestaBorrowTicker, "Lines skipped: ",skipValue, "records collected: ", vestaBorrowListLength, " - latest recordID: ",vestaBorrow, " ", recordID , " - vestaBorrows DB length: ", vestaBorrowsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", vestaBorrowTicker, "records collected: ", vestaBorrowListLength, " - userdb length: ", vestaBorrowsDbLength)
    vestaBorrowTicker = vestaBorrowTicker+1

    if daysAgo>daysLimit: exit =True
    if vestaBorrowListLength<1000: exit =True

#vestaBorrowsDbTimed.columns = vestaBorrowsDbTimed.columns.str.replace("borrows", "transaction")
vestaBorrowsDb['event']='borrow'
print("Done. Total Records collected: ", vestaBorrowsDbLength)

,troveChanges_id,troveChanges_asset_id,troveChanges_asset_name,troveChanges_transaction_id,troveChanges_transaction_blockNumber,troveChanges_transaction_timestamp,troveChanges_trove_id,troveChanges_trove_owner_id,troveChanges_trove_owner_collSurplus,troveChanges_trove_owner_totalDebt,...,troveChanges_redemption_id,troveChanges_redemption_asset_id,troveChanges_redemption_sequenceNumber,troveChanges_redemption_transaction_id,troveChanges_redemption_redeemer_id,troveChanges_redemption_tokensAttemptedToRedeem,troveChanges_redemption_tokensActuallyRedeemed,troveChanges_redemption_collateralRedeemed,troveChanges_redemption_partial,troveChanges_redemption_fee
0,9966,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0x21da8f0687a20a391e7e08c8af41efd288af11da3881...,11844766,1652312514,0x20ebe30b746fc22803dc5a8516deaae07dddbb76-0x8...,0x20ebe30b746fc22803dc5a8516deaae07dddbb76,0.0,0.000000,...,None,None,None,None,None,None,None,None,None,None
1,9885,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0x6eb09fc15a22dde0c02c9315df974ca4909a9c0504dd...,11830945,1652300970,0x8d7a5fedf55b68625b2e5953f8203b029cb9c44e-0x8...,0x8d7a5fedf55b68625b2e5953f8203b029cb9c44e,0.0,0.000000,...,None,None,None,None,None,None,None,None,None,None
2,9869,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0xbba697bae9434db16bb2eab0d06f42fb4939d6914a0e...,11828575,1652299095,0x8d7a5fedf55b68625b2e5953f8203b029cb9c44e-0x8...,0x8d7a5fedf55b68625b2e5953f8203b029cb9c44e,0.0,0.000000,...,None,None,None,None,None,None,None,None,None,None
3,9863,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0xfb753ad7db82519bb79a842b5f2c4e073af0eb7dfb42...,11826567,1652297568,0x8d7a5fedf55b68625b2e5953f8203b029cb9c44e-0x8...,0x8d7a5fedf55b68625b2e5953f8203b029cb9c44e,0.0,0.000000,...,None,None,None,None,None,None,None,None,None,None
4,9859,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0x696187a9603a26d90f165f35f6e9884ee39c07fc1961...,11824786,1652295858,0xc270b3a8428b222905c2564b0118fb4a0e10bb81-0x8...,0xc270b3a8428b222905c2564b0118fb4a0e10bb81,0.0,0.000000,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
880,100617,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0x12ef8e94f7f8e82a36ac1a2a981faaeacc8ac40518b4...,34764377,1667412994,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0-0x8...,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,0.0,0.000000,...,None,None,None,None,None,None,None,None,None,None
881,100431,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0xd90e9faf50db833330f8a87e97300bdb069bc59a8e41...,34742413,1667406070,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0-0x8...,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,0.0,0.000000,...,None,None,None,None,None,None,None,None,None,None
882,100315,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0x71c87965ad97ea3df6559016b90173a23d58093aa122...,34734210,1667403622,0x15bae977db3b62ceb5a67921fae589fd80624e8d-0x8...,0x15bae977db3b62ceb5a67921fae589fd80624e8d,0.0,550000.374479,...,None,None,None,None,None,None,None,None,None,None
883,100307,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0x5d3e659efde12edf9094644cd188263d625c12d807b3...,34733846,1667403506,0xea17e11fa9d3d772eee6f34a03f0ef44db395720-0x8...,0xea17e11fa9d3d772eee6f34a03f0ef44db395720,0.0,503318.130191,...,None,None,None,None,None,None,None,None,None,None


In [71]:
vestaBorrowsDbTimed = db.query("select "
               "    * "
                    " ,to_timestamp(borrows_timestamp::int) dateTime "
               "From vestaBorrowsDb "
               #"where lower(liquidityPools_name) like lower('%olympus%') "
                #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
               #" limit 100 "
               ).df()

vestaBorrowsDbTimed.head(100)

iterations:  0 Lines skipped:  0 records collected:  885  - latest recordID:  0x21da8f0687a20a391e7e08c8af41efd288af11da388165d1fe8423ff8468ed59   9966  - vestaBorrows DB length:  885  - latest record from:  2022-05-11 16:41:54 278 days ago
Done. Total Records collected:  885


In [83]:
vestaBorrowsDbTimedLength = len(vestaBorrowsDbTimed)
path =file+'/vesta_borrow_events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(vestaBorrowsDbTimedLength)+'_records.csv'
vestaBorrowsDbTimed.to_csv(path, index = False)

path =file+'/vesta_borrow_events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(vestaBorrowsDbTimedLength)+'_records_piped.csv'
vestaBorrowsDbTimed.to_csv(path, index = False, sep='|')
print('file saved')

,troveChanges_id,troveChanges_asset_id,troveChanges_asset_name,troveChanges_transaction_id,troveChanges_transaction_timestamp,troveChanges_transaction_blockNumber,troveChanges_trove_id,troveChanges_trove_owner_id,troveChanges_trove_owner_collSurplus,troveChanges_trove_owner_totalDebt,...,troveChanges_redemption_sequenceNumber,troveChanges_redemption_transaction_id,troveChanges_redemption_redeemer_id,troveChanges_redemption_tokensAttemptedToRedeem,troveChanges_redemption_tokensActuallyRedeemed,troveChanges_redemption_collateralRedeemed,troveChanges_redemption_partial,troveChanges_redemption_fee,event,dateTime
0,9966,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0x21da8f0687a20a391e7e08c8af41efd288af11da3881...,1652312514,11844766,0x20ebe30b746fc22803dc5a8516deaae07dddbb76-0x8...,0x20ebe30b746fc22803dc5a8516deaae07dddbb76,0.0,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,borrow,2022-05-11 23:41:54
1,9885,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0x6eb09fc15a22dde0c02c9315df974ca4909a9c0504dd...,1652300970,11830945,0x8d7a5fedf55b68625b2e5953f8203b029cb9c44e-0x8...,0x8d7a5fedf55b68625b2e5953f8203b029cb9c44e,0.0,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,borrow,2022-05-11 20:29:30
2,9869,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0xbba697bae9434db16bb2eab0d06f42fb4939d6914a0e...,1652299095,11828575,0x8d7a5fedf55b68625b2e5953f8203b029cb9c44e-0x8...,0x8d7a5fedf55b68625b2e5953f8203b029cb9c44e,0.0,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,borrow,2022-05-11 19:58:15
3,9863,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0xfb753ad7db82519bb79a842b5f2c4e073af0eb7dfb42...,1652297568,11826567,0x8d7a5fedf55b68625b2e5953f8203b029cb9c44e-0x8...,0x8d7a5fedf55b68625b2e5953f8203b029cb9c44e,0.0,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,borrow,2022-05-11 19:32:48
4,9859,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0x696187a9603a26d90f165f35f6e9884ee39c07fc1961...,1652295858,11824786,0xc270b3a8428b222905c2564b0118fb4a0e10bb81-0x8...,0xc270b3a8428b222905c2564b0118fb4a0e10bb81,0.0,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,borrow,2022-05-11 19:04:18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,68041,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0x2a70d879ac217ea6a2ef97da16bd4edf5de0183a3189...,1666382989,31596352,0xab9e8f7dcf82e40cbe183784dcf3e223f2bf130f-0x8...,0xab9e8f7dcf82e40cbe183784dcf3e223f2bf130f,0.0,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,borrow,2022-10-21 20:09:49
96,6784,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0x3c28a132a3ab4e8387b18fce535bf6d878d780bff1fc...,1648757144,8994267,0x3511950d20eab901343a13bc1263256d8ceee19b-0x8...,0x3511950d20eab901343a13bc1263256d8ceee19b,0.0,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,borrow,2022-03-31 20:05:44
97,6746,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0xbe9120fbe2e6a489e08b5182f9f9e8bad80335289ea7...,1648680386,8936313,0xf597c6452ad43651e8e6b1d35abf9e383d985e0b-0x8...,0xf597c6452ad43651e8e6b1d35abf9e383d985e0b,0.0,37170.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,borrow,2022-03-30 22:46:26
98,6717,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0x578b7c4237a3cbaff8eea3030bdfeb4c385a2542e226...,1648662324,8921267,0xedb75ba1d503a34a7c7f71c853924f4e29054f57-0x8...,0xedb75ba1d503a34a7c7f71c853924f4e29054f57,0.0,9952.83094,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,borrow,2022-03-30 17:45:24


In [73]:
#this tracks the events for vestaDepositing gOHM for lending 
vestaDepositTicker = 0
vestaDepositslist = pd.DataFrame()
vestaDepositsDb = pd.DataFrame()
vestaDepositListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (vestaDepositTicker)*(1000)
    vestaDeposits = vesta.Query.deposits(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1' } #gOHM market

    )

    vestaDepositslist = sg.query_df([
        vestaDeposits.id
        ,vestaDeposits.hash
        ,vestaDeposits.logIndex
        ,vestaDeposits.blockNumber
        ,vestaDeposits.timestamp
        ,vestaDeposits.account
        ,vestaDeposits.market
        ,vestaDeposits.market.rates
        ,vestaDeposits.asset
        ,vestaDeposits.amount
        ,vestaDeposits.amountUSD
        ,vestaDeposits.position
    ])

    #vestaDepositslist.sort_values(by=['vestaDeposits_timestamp'])

    vestaDepositsDb=pd.concat([vestaDepositsDb, vestaDepositslist])
    recordTimestamp1 = vestaDepositsDb.iat[vestaDepositTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    vestaDepositsDbLength = len(vestaDepositsDb)
    vestaDepositListLength = len(vestaDepositslist)
    recordID = vestaDepositsDb.iat[vestaDepositTicker,0]
    vestaDeposit = vestaDepositsDb.iat[vestaDepositTicker,1]
    print("iterations: ", vestaDepositTicker, "Lines skipped: ",skipValue, "records collected: ", vestaDepositListLength, " - latest recordID: ",vestaDeposit, " ", recordID , " - vestaDeposits DB length: ", vestaDepositsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", vestaDepositTicker, "records collected: ", vestaDepositListLength, " - userdb length: ", vestaDepositsDbLength)
    vestaDepositTicker = vestaDepositTicker+1
    if daysAgo>daysLimit: exit =True
    if vestaDepositListLength<1000: exit =True

#vestaDepositsDb.sort_values(by=['vestaDeposits_timestamp'])

vestaDepositsDb.columns = vestaDepositsDb.columns.str.replace("deposits", "transaction")
vestaDepositsDb['event']='deposit'
print("Done. Total Records collected: ", vestaDepositsDbLength)




file saved


In [77]:
vestaDepositsDb.head(10)

iterations:  0 Lines skipped:  0 records collected:  1000  - latest recordID:  0xfd781bb00f9e6d9599ce7aebda95979bfb902d3d73d2997bb430b7a59c911e7d   9972  - vestadeposits DB length:  1000  - latest record from:  2022-05-11 16:53:57 278 days ago
Done. Total Records collected:  1000


In [81]:
vestaDepositsDbTimed = db.query("select "
               "    * "
                    " ,to_timestamp(transaction_timestamp::int) dateTime "
               "From vestaDepositsDb "
               #"where lower(liquidityPools_name) like lower('%olympus%') "
                #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
               #" limit 100 "
               ).df()

vestaDepositsDbTimed.head(100)

,troveChanges_id,troveChanges_asset_id,troveChanges_asset_name,troveChanges_transaction_id,troveChanges_transaction_timestamp,troveChanges_transaction_blockNumber,troveChanges_trove_id,troveChanges_trove_owner_id,troveChanges_trove_owner_collSurplus,troveChanges_trove_owner_totalDebt,...,troveChanges_redemption_asset_id,troveChanges_redemption_sequenceNumber,troveChanges_redemption_transaction_id,troveChanges_redemption_redeemer_id,troveChanges_redemption_tokensAttemptedToRedeem,troveChanges_redemption_tokensActuallyRedeemed,troveChanges_redemption_collateralRedeemed,troveChanges_redemption_partial,troveChanges_redemption_fee,event
0,9972,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0xfd781bb00f9e6d9599ce7aebda95979bfb902d3d73d2...,1652313237,11845437,0x0c20e9958e4eb2d9c112d1469e17ab6c830cde09-0x8...,0x0c20e9958e4eb2d9c112d1469e17ab6c830cde09,0.0,8.976147e+03,...,None,None,None,None,None,None,None,None,None,deposit
1,9964,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0x261f7eb6da7fff00287fbd2f4cc9bc014a15cbeda432...,1652312257,11844534,0x20ebe30b746fc22803dc5a8516deaae07dddbb76-0x8...,0x20ebe30b746fc22803dc5a8516deaae07dddbb76,0.0,0.000000e+00,...,None,None,None,None,None,None,None,None,None,deposit
2,9962,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0x2fde74916c909648c83e2725fecdc902181371a58c1f...,1652309981,11842359,0xe9f6711b7c92581856e0419399bc4fb9c36ec7e3-0x8...,0xe9f6711b7c92581856e0419399bc4fb9c36ec7e3,0.0,0.000000e+00,...,None,None,None,None,None,None,None,None,None,deposit
3,9961,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0x1ebcd76a95f023492880bf7f7fbf3183e2197dd266a2...,1652309386,11841593,0x20fd6151bb6c78c6b5c89859bd5076dd852cc461-0x8...,0x20fd6151bb6c78c6b5c89859bd5076dd852cc461,0.0,0.000000e+00,...,None,None,None,None,None,None,None,None,None,deposit
4,9960,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0xc3f55ec85a72bafd3b0a10f430bc5044f274d23d50f3...,1652309304,11841518,0xa80191fca50be00f8952c69232c93d57eeacaf6f-0x8...,0xa80191fca50be00f8952c69232c93d57eeacaf6f,0.0,1.227700e+06,...,None,None,None,None,None,None,None,None,None,deposit
5,9955,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0x48627b386040057ee7a71015b67462f14104e0b757e6...,1652308553,11840674,0xc5c7e30d0df864eb281e5398416e47db5e16d887-0x8...,0xc5c7e30d0df864eb281e5398416e47db5e16d887,0.0,0.000000e+00,...,None,None,None,None,None,None,None,None,None,deposit
6,9925,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0xe4c77b55af9a7fe4742ed56f3756a20bb33c862075a8...,1652304380,11835633,0xcde1a8c256491af29eed478abf76996b0497a90f-0x8...,0xcde1a8c256491af29eed478abf76996b0497a90f,0.0,0.000000e+00,...,None,None,None,None,None,None,None,None,None,deposit
7,9924,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0x6a478fd975875045387ad810583e8270a25530599a6d...,1652304099,11835231,0x6688b44ff874e246c441397d60fc169d4972a1ab-0x8...,0x6688b44ff874e246c441397d60fc169d4972a1ab,0.0,2.200700e+04,...,None,None,None,None,None,None,None,None,None,deposit
8,99043,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0x24f693681f80f12946d463995c424c41f37a01dbd227...,1667374762,34647273,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0-0x8...,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,0.0,0.000000e+00,...,None,None,None,None,None,None,None,None,None,deposit
9,9896,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0xdf29dc3054d53c0c8681ba1e5a9ab72d0dafbcf77de4...,1652301736,11831764,0x7ff15d6cf1a7bf0efa7435543cf086835fc80d53-0x8...,0x7ff15d6cf1a7bf0efa7435543cf086835fc80d53,0.0,3.104000e+03,...,None,None,None,None,None,None,None,None,None,deposit


In [84]:
vestaDepositsDbTimedLength = len(vestaDepositsDbTimed)
path =file+'/vesta_deposit_events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(vestaDepositsDbTimedLength)+'_records.csv'
vestaDepositsDbTimed.to_csv(path, index = False)

path =file+'/vesta_deposit_events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(vestaDepositsDbTimedLength)+'_records_piped.csv'
vestaDepositsDbTimed.to_csv(path, index = False, sep='|')
print('file saved')

,troveChanges_id,troveChanges_asset_id,troveChanges_asset_name,troveChanges_transaction_id,troveChanges_transaction_timestamp,troveChanges_transaction_blockNumber,troveChanges_trove_id,troveChanges_trove_owner_id,troveChanges_trove_owner_collSurplus,troveChanges_trove_owner_totalDebt,...,troveChanges_redemption_sequenceNumber,troveChanges_redemption_transaction_id,troveChanges_redemption_redeemer_id,troveChanges_redemption_tokensAttemptedToRedeem,troveChanges_redemption_tokensActuallyRedeemed,troveChanges_redemption_collateralRedeemed,troveChanges_redemption_partial,troveChanges_redemption_fee,event,dateTime
0,9972,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0xfd781bb00f9e6d9599ce7aebda95979bfb902d3d73d2...,1652313237,11845437,0x0c20e9958e4eb2d9c112d1469e17ab6c830cde09-0x8...,0x0c20e9958e4eb2d9c112d1469e17ab6c830cde09,0.0,8.976147e+03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,deposit,2022-05-11 23:53:57
1,9964,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0x261f7eb6da7fff00287fbd2f4cc9bc014a15cbeda432...,1652312257,11844534,0x20ebe30b746fc22803dc5a8516deaae07dddbb76-0x8...,0x20ebe30b746fc22803dc5a8516deaae07dddbb76,0.0,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,deposit,2022-05-11 23:37:37
2,9962,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0x2fde74916c909648c83e2725fecdc902181371a58c1f...,1652309981,11842359,0xe9f6711b7c92581856e0419399bc4fb9c36ec7e3-0x8...,0xe9f6711b7c92581856e0419399bc4fb9c36ec7e3,0.0,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,deposit,2022-05-11 22:59:41
3,9961,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0x1ebcd76a95f023492880bf7f7fbf3183e2197dd266a2...,1652309386,11841593,0x20fd6151bb6c78c6b5c89859bd5076dd852cc461-0x8...,0x20fd6151bb6c78c6b5c89859bd5076dd852cc461,0.0,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,deposit,2022-05-11 22:49:46
4,9960,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0xc3f55ec85a72bafd3b0a10f430bc5044f274d23d50f3...,1652309304,11841518,0xa80191fca50be00f8952c69232c93d57eeacaf6f-0x8...,0xa80191fca50be00f8952c69232c93d57eeacaf6f,0.0,1.227700e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,deposit,2022-05-11 22:48:24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,8897,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0xbe2e5ec53e56702f31509ef61c623b424bc75b461428...,1651745092,11240667,0x63dcafd6b547e7cfb2e67386db3d06d67aa71c49-0x8...,0x63dcafd6b547e7cfb2e67386db3d06d67aa71c49,0.0,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,deposit,2022-05-05 10:04:52
96,8891,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0x7e0310e3991b636edef9a3eb405462e0453468fe3080...,1651732744,11227578,0xee85ae5daab560731243403f56c51654def8bc6e-0x8...,0xee85ae5daab560731243403f56c51654def8bc6e,0.0,9.502000e+03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,deposit,2022-05-05 06:39:04
97,8871,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0x421d848b7b05403b1068e3ff4d1683f17a07bd56e4c5...,1651694300,11202408,0x7b728979872ad3a870a006921c9cdffdf8f67b44-0x8...,0x7b728979872ad3a870a006921c9cdffdf8f67b44,0.0,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,deposit,2022-05-04 19:58:20
98,8855,0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1,gOHM,0x858f1980a859b84960e3b06d8d823b01f1871d090550...,1651681752,11189252,0x6d40b9be13ef1c4970df10068893371b86bd2e22-0x8...,0x6d40b9be13ef1c4970df10068893371b86bd2e22,0.0,5.827507e+03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,deposit,2022-05-04 16:29:12


In [85]:
##this tracks the ENTIRE list of events for vestaRepaying the VST vestaRepayed against gOHM
vestaRepayTicker = 0
vestaRepayslist = pd.DataFrame()
vestaRepaysDb = pd.DataFrame()
vestaRepayListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (vestaRepayTicker)*(1000)
    vestaRepays = vesta.Query.repays(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1' } #gOHM market

    )

    vestaRepayslist = sg.query_df([
        vestaRepays.id
        ,vestaRepays.hash
        ,vestaRepays.logIndex
        ,vestaRepays.blockNumber
        ,vestaRepays.timestamp
        ,vestaRepays.account
        ,vestaRepays.market
        ,vestaRepays.market.rates
        ,vestaRepays.asset
        ,vestaRepays.amount
        ,vestaRepays.amountUSD
        ,vestaRepays.position
    ])

    #vestaRepayslist.sort_values(by=['vestaRepays_timestamp'])

    vestaRepaysDb=pd.concat([vestaRepaysDb, vestaRepayslist])
    recordTimestamp1 = vestaRepaysDb.iat[vestaRepayTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    vestaRepaysDbLength = len(vestaRepaysDb)
    vestaRepayListLength = len(vestaRepayslist)
    recordID = vestaRepaysDb.iat[vestaRepayTicker,0]
    vestaRepay = vestaRepaysDb.iat[vestaRepayTicker,1]
    print("iterations: ", vestaRepayTicker, "Lines skipped: ",skipValue, "records collected: ", vestaRepayListLength, " - latest recordID: ",vestaRepay, " ", recordID , " - vestaRepays DB length: ", vestaRepaysDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", vestaRepayTicker, "records collected: ", vestaRepayListLength, " - userdb length: ", vestaRepaysDbLength)
    vestaRepayTicker = vestaRepayTicker+1

    if daysAgo>daysLimit: exit =True
    if vestaRepayListLength<1000: exit =True

vestaRepaysDb.columns = vestaRepaysDb.columns.str.replace("repays", "transaction")
vestaRepaysDb['event']='Repay'
print("Done. Total Records collected: ", vestaRepaysDbLength)

file saved


In [31]:
vestaRepaysDb.head(10)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_timestampClosed,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event
0,repay-0xac8ecdaf59200e43e6e6233bcd31728b0db4bc...,0xac8ecdaf59200e43e6e6233bcd31728b0db4bc543593...,26,60476974,1676271078,0x6c01ec9622e50799d62ca076f2e2aa34dc840d5b,4,0,4,1,...,1.676271e+09,BORROWER,None,0,0,0,2,2,0,Repay
1,repay-0x9633a7166f3744cde872b630378d9eaeec5f0d...,0x9633a7166f3744cde872b630378d9eaeec5f0da82d73...,26,60475459,1676270640,0x5c9d09716404556646b0b4567cb4621c18581f94,5,0,5,1,...,1.676271e+09,BORROWER,None,0,0,0,2,3,0,Repay
2,repay-0x851007fa5a8371105a73452ccbd7a0a4eb976e...,0x851007fa5a8371105a73452ccbd7a0a4eb976e7d2c12...,32,59865436,1676083788,0xc9e166f2737e004e93800dcef0d608b014df07f6,25,4,21,16,...,NaN,BORROWER,None,3515000787053571080985,0,0,30,21,0,Repay
3,repay-0xa243d87a35b20328eac5637a6db98058707cd0...,0xa243d87a35b20328eac5637a6db98058707cd0161df6...,61,59508912,1675978810,0xf5d0318692f822680b733955828a25507f031c95,2,0,2,1,...,1.675979e+09,BORROWER,None,0,0,0,1,1,0,Repay
4,repay-0xdee0c294dd70fb50547428f7e6d6ea6b135cb8...,0xdee0c294dd70fb50547428f7e6d6ea6b135cb8a4d75a...,33,59130546,1675866794,0x515755b2c5a209976cf0de869c30f45ac7495a60,6,0,6,12,...,1.675867e+09,BORROWER,None,0,0,0,1,1,0,Repay
5,repay-0xd6e3c80447789b02b1b9282c1624d5885a19f4...,0xd6e3c80447789b02b1b9282c1624d5885a19f40712bd...,25,59010222,1675835228,0x7ee54ab0f204bb3a83df90fdd824d8b4abe93222,3,0,3,1,...,1.675835e+09,BORROWER,None,0,0,0,1,3,0,Repay
6,repay-0xae8658b03d6aee809d73702ece0d60ced1b6be...,0xae8658b03d6aee809d73702ece0d60ced1b6be240b82...,58,58773043,1675767985,0x09fc8cd1d422e77cef172ef85cda1953915a8b74,2,0,2,1,...,1.675768e+09,BORROWER,None,0,0,0,1,1,0,Repay
7,repay-0xa02ca5cb2651d3bd78de426c9c010eea013be5...,0xa02ca5cb2651d3bd78de426c9c010eea013be53c8843...,23,58627462,1675722780,0xfa4dfbe7b012b1d92fdd5544f30518bb58bd41ce,2,0,2,1,...,1.675723e+09,BORROWER,None,0,0,0,1,2,0,Repay
8,repay-0x44899a838a0d9eaf2f48a95594a3b1041bcbdf...,0x44899a838a0d9eaf2f48a95594a3b1041bcbdfabe482...,29,58627282,1675722712,0xfa4dfbe7b012b1d92fdd5544f30518bb58bd41ce,2,0,2,1,...,1.675723e+09,BORROWER,None,0,0,0,1,2,0,Repay
9,repay-0x91f5c48839b978c11fbbe3f9502c5538060257...,0x91f5c48839b978c11fbbe3f9502c55380602570911ec...,29,58336925,1675634888,0xe2a4de267cdd4ff5ed9ba13552f5c624b12db9b2,20,6,14,65,...,NaN,BORROWER,None,78903329999999999121255,0,0,6,1,0,Repay


In [88]:
vestaRepaysDbTimed = db.query("select "
               "    * "
                    " ,to_timestamp(transaction_timestamp::int) dateTime "
               "From vestaRepaysDb "
               #"where lower(liquidityPools_name) like lower('%olympus%') "
                #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
               #" limit 100 "
               ).df()

vestaRepaysDbTimed.head(100)

In [89]:
vestaRepaysDbTimedLength=len(vestaRepaysDbTimed)
path =file+'/vesta_repay_events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(vestaRepaysDbTimedLength)+'_records.csv'
vestaRepaysDbTimed.to_csv(path, index = False)

path =file+'/vesta_repay_events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(vestaRepaysDbTimedLength)+'_records_piped.csv'
vestaRepaysDbTimed.to_csv(path, index = False, sep='|')
print('file saved')

iterations:  0 Lines skipped:  0 records collected:  1000  - latest recordID:  borrowAsset   0x023999891639c1c93db864ce955525a0804c6f5216d81d75d341c81be072987a  - actions DB length:  1000  - latest record from:  2023-02-13 13:52:23 0 days ago
iterations:  1 Lines skipped:  1000 records collected:  1000  - latest recordID:  borrowAsset   0xd27c863004571db8166a41942a3e565e1cb4ed6392a1233723f6ba6cfb681d91  - actions DB length:  2000  - latest record from:  2023-02-13 10:52:11 0 days ago
iterations:  2 Lines skipped:  2000 records collected:  1000  - latest recordID:  addCollateral   0x93bfdc153c70c47dc88df5b2c7a28fa129c178d3c2691612c7d08ac1e0c70467  - actions DB length:  3000  - latest record from:  2023-02-13 10:33:35 0 days ago
iterations:  3 Lines skipped:  3000 records collected:  1000  - latest recordID:  borrowAsset   0xf6aae4916b2c041fc5f24c61ac3ee16ea860b2c7f9f271eafeb59f588eb0d0ab  - actions DB length:  4000  - latest record from:  2023-02-13 08:42:35 0 days ago
iterations:  4 Li

In [125]:
vestaBorrowDepositEventsDb = pd.DataFrame()
vestaBorrowDepositEventsDb=pd.concat([vestaDepositsDbTimed, vestaBorrowsDbTimed])#,vestaRepaysDbTimed])
vestaBorrowDepositEventsDbLength=len(vestaBorrowDepositEventsDb)
vestaBorrowDepositEventsDb.head(10)

,actions_id,actions_type,actions_amount,actions_timestamp,actions_block,actions_token_name,actions_root_address,dateTime,amountUsd
0,0x023999891639c1c93db864ce955525a0804c6f5216d8...,borrowAsset,14000000000000000000000,1676325143,16622639,Frax,0x1effd55a8646f7dc67c7578c20ce575cefeb1120,2023-02-13 21:52:23,14000.000000
1,0xd27c863004571db8166a41942a3e565e1cb4ed6392a1...,borrowAsset,20000000000000000000000,1676314331,16621741,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,2023-02-13 18:52:11,20000.000000
2,0x93bfdc153c70c47dc88df5b2c7a28fa129c178d3c269...,addCollateral,261775476655574434,1676313215,16621648,Governance OHM,0x440c8ab91ba7c8763b3d9b3357dc1c0d13a148aa,2023-02-13 18:33:35,0.261775
3,0xf6aae4916b2c041fc5f24c61ac3ee16ea860b2c7f9f2...,borrowAsset,10000000000000000000000,1676306555,16621099,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,2023-02-13 16:42:35,10000.000000
4,0x186f6c99d4fe58a91ba6b8c57bfdde7e40b3bc9137a1...,borrowAsset,10000000000000000000000,1676298455,16620432,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,2023-02-13 14:27:35,10000.000000
5,0xcf9fdd868da296feca4c40710bd7601676265a61f3bd...,removeCollateral,1812153561190806971,1676292563,16619943,Governance OHM,0xc0b1f200aa9546ce074397bed7438bb2ec033877,2023-02-13 12:49:23,1.812154
6,0x1ec1f1d781c20ba212b8c86e9141634dfa5ab919c9b4...,repayAsset,3452642443104311600788,1676292491,16619937,Frax,0xc0b1f200aa9546ce074397bed7438bb2ec033877,2023-02-13 12:48:11,3452.642443
7,0xa78cd3b8692a61e7e2b1781c7e7bc2e46878b4e81597...,removeCollateral,1302696019236763758,1676292119,16619906,Governance OHM,0xc0b1f200aa9546ce074397bed7438bb2ec033877,2023-02-13 12:41:59,1.302696
8,0x395e8e7df1cfc36cf1c5c5e745ac1fcb7d166a1b66b3...,repayAsset,2482001407422899262526,1676292095,16619904,Frax,0xc0b1f200aa9546ce074397bed7438bb2ec033877,2023-02-13 12:41:35,2482.001407
9,0x03d12a970c30f0e7353660d5c58b307c6e5be4a87e40...,removeCollateral,926234315317722764,1676291219,16619832,Governance OHM,0xc0b1f200aa9546ce074397bed7438bb2ec033877,2023-02-13 12:26:59,0.926234


In [126]:
vestaBorrowDepositEventsDbLength = len(vestaBorrowDepositEventsDb)
path =file+'/vesta_BorrowDeposit_Events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(vestaBorrowDepositEventsDbLength)+'_records.csv'
vestaBorrowDepositEventsDb.to_csv(path, index = False)

path =file+'/vesta_BorrowDeposit_Events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(vestaBorrowDepositEventsDbLength)+'_records_piped.csv'
vestaBorrowDepositEventsDb.to_csv(path, index = False, sep='|')
print('file saved')

file saved


In [127]:
#let's pull interest rate data
interestRateTicker = 0
interestRateslist = pd.DataFrame()
interestRatesDb = pd.DataFrame()
interestRateListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (interestRateTicker)*(1000)
    interestRates = vesta.Query.interestRates(
        orderBy='id',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        #where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' } #gOHM market

    )

    interestRateslist = sg.query_df([
        interestRates.id
        ,interestRates.rate #5.21% should be stored as 5.21
        ,interestRates.duration #in days
        ,interestRates.maturityBlock #Maturity of the loan in block height. Only applies to fixed term lending (e.g. Notional)
        ,interestRates.side #The party the interest is paid to / received from
        ,interestRates.type #The type of interest rate (e.g. stable, fixed, variable, etc)
    ])

    #interestRateslist.sort_values(by=['interestRates_timestamp'])

    interestRatesDb=pd.concat([interestRateslist, interestRatesDb])
    interestRatesDbLength = len(interestRatesDb)
    interestRateListLength = len(interestRateslist)
    print("iterations: ", interestRateTicker, "Lines skipped: ",skipValue, "records collected: ", interestRateListLength)
    #print("iterations: ", interestRateTicker, "records collected: ", interestRateListLength, " - userdb length: ", interestRatesDbLength)
    interestRateTicker = interestRateTicker+1
    if interestRateListLength<1000: exit =True

print("Done. Total Records collected: ", interestRatesDbLength)




,actions_root_address,dateTime,amountUsd,actions_token_name
0,0x233e3017447e66d91e2cd969233adc5bb0524ec1,2022-12-15 16:28:35,3000.0,Frax
1,0x639022553eba685dad01121b6eb448fed97517f4,2022-12-15 08:33:47,0.0,Frax
2,0x9adcdc109dc88adc2e1493829e1bdf59635a4692,2022-12-15 00:15:23,10000.0,Frax
3,0xca5df910f3e2226324707efe7fa6d03a0f821353,2022-12-14 22:59:35,67000.0,Frax
4,0xca5df910f3e2226324707efe7fa6d03a0f821353,2022-12-14 22:53:35,83000.0,Frax
5,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,2022-12-14 15:54:11,37000.0,Frax
6,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,2022-12-14 14:37:23,55000.0,Frax
7,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,2022-12-14 14:26:23,52001.0,Frax
8,0xf10918fc1725a65960db2b906b4868a0fdbb6870,2022-12-13 18:42:35,40000.0,Frax
9,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,2022-12-13 15:06:23,0.0,Frax


In [131]:
interestRatesDb.head(5)

SyntaxError: unterminated string literal (detected at line 2) (4170740533.py, line 2)

In [137]:
interestRatesDbLength = len(interestRatesDb)
path =file+'/vesta_interest_rates_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(interestRatesDbLength)+'_records.csv'
interestRatesDb.to_csv(path, index = False)

path =file+'/vesta_interest_rates_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(interestRatesDbLength)+'_records_piped.csv'
interestRatesDb.to_csv(path, index = False, sep='|')
print('file saved')

,calDay,amountUsdBorrowed,borrowers,protocol
0,2023-02-13,26298000.0,2,Frax
1,2023-02-12,12662000.0,2,Frax
2,2023-02-11,23132500.0,2,Frax
3,2023-02-10,12175000.0,1,Frax
4,2023-02-09,91361200.0,5,Frax
5,2023-02-08,46265000.0,2,Frax
6,2023-02-07,19747850.0,3,Frax
7,2023-02-06,0.0,1,Frax
8,2023-02-05,65745000.0,1,Frax
9,2023-02-03,4870000.0,1,Frax


##this WILL Capture the list of people who lended gOHM for others to borrow
lendTicker = 0
lendslist = pd.DataFrame()
lendsDb = pd.DataFrame()
lendListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (lendTicker)*(1000)
    lends = silo.Query.lends(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' } #gOHM market

    )

    lendslist = sg.query_df([
        lends.id
        ,lends.hash
        ,lends.logIndex
        ,lends.blockNumber
        ,lends.timestamp
        ,lends.account
        ,lends.market
        ,lends.asset
        ,lends.amount
        ,lends.amountUSD
    ])

    #lendslist.sort_values(by=['lends_timestamp'])

    lendsDb=pd.concat([lendsDb, lendslist])
    recordTimestamp1 = lendsDb.iat[lendTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    lendsDbLength = len(lendsDb)
    lendListLength = len(lendslist)
    recordID = lendsDb.iat[lendTicker,0]
    lend = lendsDb.iat[lendTicker,1]
    print("iterations: ", lendTicker, "Lines skipped: ",skipValue, "records collected: ", lendListLength, " - latest recordID: ",lend, " ", recordID , " - lends DB length: ", lendsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", lendTicker, "records collected: ", lendListLength, " - userdb length: ", lendsDbLength)
    lendTicker = lendTicker+1
    if daysAgo>daysLimit: exit =True
    if lendListLength<1000 exit =True

#lendsDb.sort_values(by=['lends_timestamp'])
print("Done. Total Records collected: ", lendsDbLength)

************************************************************
FINE.... FRAX TOO

This subgraph is completely different, so we're going to  have to crunch in order to merge all these tables into coherent reports

In [138]:
frax = sg.load_api('https://gateway.thegraph.com/api/'+apiKey+'/subgraphs/id/GELTrjPJYEzxyp6Y2CtEaYpGHcJNrJA6i5Ci4KfJSEsf')

,calDay,amountUsdBorrowed,borrowers,protocol
0,2022-05-11,34625.0,3,Vesta
1,2022-11-01,34363.0,2,Vesta
2,2022-10-31,26327.0,1,Vesta
3,2022-05-09,192375.0,1,Vesta
4,2022-05-08,116344.0,2,Vesta
5,2022-10-30,16459.0,2,Vesta
6,2022-05-06,513.0,1,Vesta
7,2022-05-05,4975.0,1,Vesta
8,2022-05-04,5994.0,2,Vesta
9,2022-10-29,20270.0,1,Vesta


In [139]:
actionListLength=0
actionTicker = 0
fraxUsersDb = pd.DataFrame()
actionsDb = pd.DataFrame()
actionListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    #listLength=0
    skipValue = (actionTicker)*(1000)
    actionsQuery = frax.Query.actions(
         orderBy='timestamp'
        ,orderDirection='desc'
        ,first=1000
        ,skipValue = skipValue
        #,where = [frax.Query.actions.token.id == '0x66bf36dba79d4606039f04b32946a260bcd3ff52']
        ,where={ 'pair': '0x66bf36dba79d4606039f04b32946a260bcd3ff52' }
        #,where = {'positions.pairid':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}
    #{'users_positions_pair_id':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}

    )
    actions= sg.query_df([
        actionsQuery.id
        ,actionsQuery.type
        ,actionsQuery.amount
        ,actionsQuery.timestamp
        ,actionsQuery.block
        ,actionsQuery.token.name
        ,actionsQuery.root.address
        ]
    )

    actions.sort_values(by=['actions_timestamp'])

    actionsDb=pd.concat([actionsDb, actions])
    recordTimestamp1 = actionsDb.iat[actionTicker,3]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    actionsDbLength = len(actionsDb)
    actionListLength = len(actions)
    recordID = actionsDb.iat[actionTicker,0]
    action = actionsDb.iat[actionTicker,1]
    print("iterations: ", actionTicker, "Lines skipped: ",skipValue, "records collected: ", actionListLength, " - latest recordID: ",action, " ", recordID , " - actions DB length: ", actionsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", actionTicker, "records collected: ", actionListLength, " - userdb length: ", actionsDbLength)
    if daysAgo>daysLimit: exit =True
    if actionsDbLength<1000: exit =True
    actionTicker = actionTicker+1
print("Done. Total Records collected: ", actionsDbLength)

,calDay,amountUsdBorrowed,borrowers,protocol
0,2022-05-11,3.462500e+04,3,Vesta
1,2022-11-01,3.436300e+04,2,Vesta
2,2022-10-31,2.632700e+04,1,Vesta
3,2022-05-09,1.923750e+05,1,Vesta
4,2022-05-08,1.163440e+05,2,Vesta
...,...,...,...,...
65,2022-12-07,1.004438e+08,3,Frax
66,2022-12-06,2.435000e+06,1,Frax
67,2022-12-05,2.478505e+08,5,Frax
68,2022-12-04,1.681754e+08,7,Frax


In [140]:
    actionsDbTimed = db.query("select "
                   "    * "
                        " ,to_timestamp(actions_timestamp) dateTime "
                    ",(actions_amount::hugeint*1e-18) amountUsd "
                   "From actionsDb "
                   #"where lower(liquidityPools_name) like lower('%olympus%') "
                    #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
                   #" limit 100 "
                   ).df()
actionsDbTimed.head(10)

file saved


In [42]:
actionsDbTimedLength = len(actionsDbTimed)
path =file+'/frax_actions_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(actionsDbTimedLength)+'_records.csv'
actionsDbTimed.to_csv(path, index = False)

path =file+'/frax_actions_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(actionsDbTimedLength)+'_records_piped.csv'
actionsDbTimed.to_csv(path, index = False, sep='|')
print('file saved')

file saved


In [43]:
fraxBorrowsDb = db.query("select distinct"
                   "    actions_root_address "
                         ", dateTime "
                         ", amountUSD "
                         ", actions_token_name "
                       "From actionsDbTimed "
                       "where actions_type= 'borrowAsset' "
                   ).df()
fraxBorrowsDb.head(10)

,actions_root_address,dateTime,amountUsd,actions_token_name
0,0x233e3017447e66d91e2cd969233adc5bb0524ec1,2022-12-15 16:28:35,3000.0,Frax
1,0x639022553eba685dad01121b6eb448fed97517f4,2022-12-15 08:33:47,0.0,Frax
2,0x9adcdc109dc88adc2e1493829e1bdf59635a4692,2022-12-15 00:15:23,10000.0,Frax
3,0xca5df910f3e2226324707efe7fa6d03a0f821353,2022-12-14 22:59:35,67000.0,Frax
4,0xca5df910f3e2226324707efe7fa6d03a0f821353,2022-12-14 22:53:35,83000.0,Frax
5,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,2022-12-14 15:54:11,37000.0,Frax
6,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,2022-12-14 14:37:23,55000.0,Frax
7,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,2022-12-14 14:26:23,52001.0,Frax
8,0xf10918fc1725a65960db2b906b4868a0fdbb6870,2022-12-13 18:42:35,40000.0,Frax
9,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,2022-12-13 15:06:23,0.0,Frax


**********************************************************
LET'S PLAY AND CRUNCH DATA - BORROWS


In [44]:
fraxBorrowsDb.drop_duplicates

fraxBorrowsDbLength = len(fraxBorrowsDb)
path =file+'/frax_borrows_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(fraxBorrowsDbLength)+'_records.csv'
fraxBorrowsDb.to_csv(path, index = False)

path =file+'/frax_borrows_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(fraxBorrowsDbLength)+'_records_piped.csv'
fraxBorrowsDb.to_csv(path, index = False, sep='|')
print('file saved')

file saved


In [45]:
fraxBorrowedUSDDaily = db.query(
                        "select "
                            "date_trunc('day',dateTime) calDay "
                            ",sum(amountUsd) amountUsdBorrowed "
                        "   ,count(distinct actions_root_address) borrowers "
                       "From fraxBorrowsDb "
                        "Group by 1 "
                        "order by 1"
                   ).df()
fraxBorrowedUSDDaily['protocol'] = 'Frax'
fraxBorrowedUSDDaily.head(10)

,calDay,amountUsdBorrowed,borrowers,protocol
0,2022-12-03,58608.786050,7,Frax
1,2022-12-04,345329.334643,7,Frax
2,2022-12-05,508933.348518,5,Frax
3,2022-12-06,5000.000000,1,Frax
4,2022-12-07,206250.000000,3,Frax
5,2022-12-08,531726.823715,7,Frax
6,2022-12-09,288250.000000,4,Frax
7,2022-12-10,160000.000000,2,Frax
8,2022-12-11,8000.000000,1,Frax
9,2022-12-12,14125.000000,4,Frax


fraxCollateralDb = db.query("select "
                   "    * "
                       "From actionsDbTimed "
                       "where actions_type= 'addCollateral' "
                   ).df()

fraxCollateralDb.head(10)

In [46]:
siloBorrowedUSDDaily = db.query(
                        "select "
                            "date_trunc('day',dateTime) calDay "
                            ",sum(transaction_amountUSD::hugeint) amountUsdBorrowed "
                        "   ,count(distinct transaction_account_id) borrowers "
                       "From borrowsDbTimed "
                        "Group by 1 "
                   ).df()
siloBorrowedUSDDaily['protocol'] = 'Silo'
siloBorrowedUSDDaily.head(10)

,calDay,amountUsdBorrowed,borrowers,protocol
0,2023-02-13,18510.0,1,Silo
1,2023-02-09,1122006.0,2,Silo
2,2023-02-08,2382.0,1,Silo
3,2023-02-07,526416.0,5,Silo
4,2023-02-01,30006.0,1,Silo
5,2023-01-28,60246.0,1,Silo
6,2022-12-17,2988.0,1,Silo
7,2022-12-13,48246.0,1,Silo
8,2022-12-12,95646.0,2,Silo
9,2022-11-23,3966.0,2,Silo


In [47]:
vestaBorrowedUSDDaily = db.query(
                        "select "
                            "date_trunc('day',dateTime) calDay "
                            ",sum(borrows_amountUSD::hugeint) amountUsdBorrowed "
                        "   ,count(distinct borrows_account_id) borrowers "
                       "From vestaBorrowsDbTimed "
                        "Group by 1 "
                   ).df()
vestaBorrowedUSDDaily['protocol'] = 'Vesta'
vestaBorrowedUSDDaily.head(10)

,calDay,amountUsdBorrowed,borrowers,protocol
0,2023-02-13,300.0,1,Vesta
1,2023-02-12,91777.0,1,Vesta
2,2023-02-11,7615.0,2,Vesta
3,2023-02-10,261.0,1,Vesta
4,2023-02-09,50100.0,2,Vesta
5,2023-02-08,25881.0,1,Vesta
6,2023-02-07,126480.0,3,Vesta
7,2023-02-06,14572.0,1,Vesta
8,2023-02-05,7583.0,3,Vesta
9,2023-02-04,2286.0,1,Vesta


In [48]:
#Don't judge if I use pandas for some things and SQL for others. I'm a noob :P
dailyUsdBorrowdDb =  pd.concat([vestaBorrowedUSDDaily, siloBorrowedUSDDaily,fraxBorrowedUSDDaily])
dailyUsdBorrowdDb.head(400)

,calDay,amountUsdBorrowed,borrowers,protocol
0,2023-02-13,300.0,1,Vesta
1,2023-02-12,91777.0,1,Vesta
2,2023-02-11,7615.0,2,Vesta
3,2023-02-10,261.0,1,Vesta
4,2023-02-09,50100.0,2,Vesta
...,...,...,...,...
65,2023-02-09,187600.0,5,Frax
66,2023-02-10,25000.0,1,Frax
67,2023-02-11,47500.0,2,Frax
68,2023-02-12,26000.0,2,Frax


In [49]:
dailyUsdBorrowdDbLength = len(dailyUsdBorrowdDb)
path =file+'/dailyUsdBorrowed_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(dailyUsdBorrowdDbLength)+'_records.csv'
dailyUsdBorrowdDb.to_csv(path, index = False)

path =file+'/dailyUsdBorrowed_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(dailyUsdBorrowdDbLength)+'_records_piped.csv'
dailyUsdBorrowdDb.to_csv(path, index = False, sep='|')
print('file saved')

file saved


In [50]:
dailyUsdBorrowdDbLength

391

*******************************8
REPAYS!

In [51]:
fraxRepaysDb = db.query("select "
                   "    * "
                    ",(actions_amount::hugeint*1e-18) amountUsdRepayed "
                       "From actionsDbTimed "
                       "where actions_type= 'repayAsset' "
                   ).df()
fraxRepaysDb.head(10)

,actions_id,actions_type,actions_amount,actions_timestamp,actions_block,actions_token_name,actions_root_address,dateTime,amountUsd,amountUsdRepayed
0,0x1ec1f1d781c20ba212b8c86e9141634dfa5ab919c9b4...,repayAsset,3452642443104311600788,1676292491,16619937,Frax,0xc0b1f200aa9546ce074397bed7438bb2ec033877,2023-02-13 12:48:11,3452.642443,3452.642443
1,0x395e8e7df1cfc36cf1c5c5e745ac1fcb7d166a1b66b3...,repayAsset,2482001407422899262526,1676292095,16619904,Frax,0xc0b1f200aa9546ce074397bed7438bb2ec033877,2023-02-13 12:41:35,2482.001407,2482.001407
2,0x61b85113554f41e41beb5a81ae45336168b6eedaf7c5...,repayAsset,1300000127635337104324,1676079827,16602316,Frax,0xf10918fc1725a65960db2b906b4868a0fdbb6870,2023-02-11 01:43:47,1300.000128,1300.000128
3,0xc7c1739911ae71f7726d1d32214edb8161a341f7fcd7...,repayAsset,812894298298405441439,1676068895,16601407,Frax,0x46535cbc3fa574c32ea392d4f89c518290597072,2023-02-10 22:41:35,812.894298,812.894298
4,0x40e939c94b527bd4eb9f8f102cfe3c179b61f0da6bb7...,repayAsset,20000000657920034222343,1676049335,16599784,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,2023-02-10 17:15:35,20000.000658,20000.000658
5,0x83e2ea08ec4f8a38735c7ad01954ce62ae9f79b84e7d...,repayAsset,66251004408366659013068,1676047871,16599664,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,2023-02-10 16:51:11,66251.004408,66251.004408
6,0x5dee647c2e8ebfad8bac45dd1ca3eab67b4841ef1692...,repayAsset,79833054078273887555863,1676032955,16598429,Frax,0x46535cbc3fa574c32ea392d4f89c518290597072,2023-02-10 12:42:35,79833.054078,79833.054078
7,0x7f9a9b4079f340cc010ddcf1ccbac3db2e1ea4e29adb...,repayAsset,79999010767531137119399,1675924787,16589468,Frax,0x46535cbc3fa574c32ea392d4f89c518290597072,2023-02-09 06:39:47,79999.010768,79999.010768
8,0xbc537609adb35fdd165f22a83da392b898080e7e2b70...,repayAsset,79999005383702015426508,1675900835,16587479,Frax,0x46535cbc3fa574c32ea392d4f89c518290597072,2023-02-09 00:00:35,79999.005384,79999.005384
9,0x87953dce0f55cdd157c52fb9ce2ede8c34012f0c2c29...,repayAsset,25222001668846023315909,1675824611,16581186,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,2023-02-08 02:50:11,25222.001669,25222.001669


In [52]:
fraxRepaiddUsdDaily = db.query(
                        "select "
                            "date_trunc('day',dateTime) calDay "
                            ",sum(amountUsdRepayed) amountUsdRepaid "
                        "   ,count(distinct actions_root_id) repayers "
                       "From fraxRepaysDb "
                        "Group by 1 "
                        "order by 1"
                   ).df()
fraxRepaiddUsdDaily['protocol'] = 'Frax'
fraxRepaiddUsdDaily.head(10)

RuntimeError: Binder Error: Referenced column "actions_root_id" not found in FROM clause!
Candidate bindings: "fraxRepaysDb.actions_id"

In [54]:
vestaUsdRepaidDaily = db.query(
                        "select "
                            "date_trunc('day',dateTime) calDay "
                            ",sum(transaction_amountUSD::hugeint) amountUsdRepaid "
                        "   ,count(distinct transaction_account_id) repayers "
                       "From  vestaRepaysDbTimed "
                        "Group by 1 "
                   ).df()
vestaUsdRepaidDaily['protocol'] = 'Vesta'
vestaUsdRepaidDaily.head(10)

,calDay,amountUsdRepaid,repayers,protocol
0,2023-02-13,2341.0,2,Vesta
1,2023-02-11,150.0,1,Vesta
2,2023-02-09,2286.0,1,Vesta
3,2023-02-08,31182.0,2,Vesta
4,2023-02-07,13173.0,1,Vesta
5,2023-02-06,1000.0,1,Vesta
6,2023-02-05,124831.0,2,Vesta
7,2023-02-02,10768.0,3,Vesta
8,2023-02-01,20270.0,2,Vesta
9,2023-01-31,1586.0,1,Vesta


In [ ]:
SiloUsdRepaidDaily = db.query(
                        "select "
                            "date_trunc('day',to_timestamp(transaction_timestamp::int)) calDay "
                            ",sum(transaction_amountUSD::hugeint) amountUsdRepaid "
                        "   ,count(distinct transaction_account_id) repayers "
                       "From  repaysDb "
                        "Group by 1 "
                   ).df()
SiloUsdRepaidDaily['protocol'] = 'Silo'
SiloUsdRepaidDaily.head(10)

*****************************
DAILY SNAPSHOTS - FRAX FIRST

In [55]:
pairSnapshotsListLength=0
pairSnapshotsTicker = 0
Snapshots = pd.DataFrame()
pairSnapshotsDb = pd.DataFrame()
pairSnapshotsListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit= False
while exit==False:
    #listLength=0
    skipValue = (pairSnapshotsTicker)*(1000)
    pairsQuery = frax.Query.pairPerDays(
         orderBy='id'
        ,orderDirection='desc'
        ,first=1000
        ,skipValue = skipValue
        #,where = [frax.Query.pairs.token.id == '0x66bf36dba79d4606039f04b32946a260bcd3ff52']
        ,where={ 'pair': '0x66bf36dba79d4606039f04b32946a260bcd3ff52' }
        #,where = {'positions.pairid':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}
    #{'users_positions_pair_id':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}

    )
    Snapshots= sg.query_df([
    pairsQuery.id
    ,pairsQuery.feeToProtocolRate
    ,pairsQuery.exchangeRate
    ,pairsQuery.totalAssetAmount
    ,pairsQuery.totalAssetShare
    ,pairsQuery.totalCollateral
    ,pairsQuery.totalBorrowAmount
    ,pairsQuery.totalBorrowShare
    ,pairsQuery.interestPerSecond
    ,pairsQuery.utilization
    ,pairsQuery.totalFeesAmount
    ,pairsQuery.totalFeesShare
    ,pairsQuery.lastAccrued
    ,pairsQuery.block
    ,pairsQuery.timestamp
    ,pairsQuery.totalAssetValue
    ,pairsQuery.totalCollateralValue
    ,pairsQuery.totalBorrowValue
    ,pairsQuery.pair
    ]
    )

    Snapshots.sort_values(by=['pairPerDays_timestamp'])

    pairSnapshotsDb=pd.concat([pairSnapshotsDb, Snapshots])
    recordTimestamp1 = pairSnapshotsDb.iat[pairSnapshotsTicker,14]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    pairSnapshotsDbLength = len(pairSnapshotsDb)
    pairSnapshotsListLength = len(Snapshots)
    recordID = pairSnapshotsDb.iat[pairSnapshotsTicker,0]
    Snapshot = pairSnapshotsDb.iat[pairSnapshotsTicker,1]
    print("iterations: ", pairSnapshotsTicker, "Lines skipped: ",skipValue, "records collected: ", pairSnapshotsListLength, " - latest recordID: ",Snapshot, " ", recordID , " - pairSnapshots DB length: ", pairSnapshotsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", pairSnapshotsTicker, "records collected: ", pairSnapshotsListLength, " - userdb length: ", pairSnapshotsDbLength)
    if daysAgo>daysLimit: exit =True
    if pairSnapshotsListLength<1000: exit =True
    pairSnapshotsTicker = pairSnapshotsTicker+1

print("Done. Total Records collected: ", pairSnapshotsDbLength)

iterations:  0 Lines skipped:  0 records collected:  77  - latest recordID:  0   0x66bf36dba79d4606039f04b32946a260bcd3ff52-Recent  - pairSnapshots DB length:  77  - latest record from:  2023-02-13 13:52:23 0 days ago
Done. Total Records collected:  77


In [56]:
fraxPairSnapshotsDbTimed= db.query("select "
                   "    * "
                        " ,to_timestamp(pairPerDays_timestamp) dateTime "
                   "From pairSnapshotsDb "
                   #"where lower(liquidityPools_name) like lower('%olympus%') "
                    #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
                   #" limit 100 "
                   ).df()
fraxPairSnapshotsDbTimed.head(1000)

,pairPerDays_id,pairPerDays_feeToProtocolRate,pairPerDays_exchangeRate,pairPerDays_totalAssetAmount,pairPerDays_totalAssetShare,pairPerDays_totalCollateral,pairPerDays_totalBorrowAmount,pairPerDays_totalBorrowShare,pairPerDays_interestPerSecond,pairPerDays_utilization,...,pairPerDays_pair_oracleMultiplyAddress_id,pairPerDays_pair_oracleDivideAddress_id,pairPerDays_pair_rateContract_id,pairPerDays_pair_maxLTV,pairPerDays_pair_liquidationFee,pairPerDays_pair_maturity,pairPerDays_pair_pauseStatus,pairPerDays_pair_lenderWhitelistActive,pairPerDays_pair_borrowerWhitelistActive,dateTime
0,0x66bf36dba79d4606039f04b32946a260bcd3ff52-Recent,0,361722603166047,5214478754084957077245512,5149945278165210104338772,2920785897200163201938,3934338506416718401226357,3873285829567710248046516,2655789209,75450,...,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False,2023-02-13 21:52:23
1,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19401,0,361722603166047,5214478754084957077245512,5149945278165210104338772,2920785897200163201938,3934338506416718401226357,3873285829567710248046516,2655789209,75450,...,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False,2023-02-13 21:52:23
2,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19400,0,378555412205106,5213593301053638152367912,5149945278165210104338772,2924565205619252920997,3885387697235926687212071,3825964183186942371065627,2625172870,74524,...,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False,2023-02-12 21:56:35
3,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19399,0,371588713173683,4912412879013939854607085,4853295245502887806145054,2895557877358142227171,3858491275196228389451244,3800357568493770528965925,2758282929,78545,...,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False,2023-02-11 13:14:23
4,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19398,0,377188472125277,4911010288245155556695770,4852807448574856557058528,2891463499070405044980,3811382372367803420789784,3754848101846442098818921,2727331303,77608,...,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False,2023-02-10 22:42:47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19330,0,416982985435782,3000265799749067769553552,2999681610314728238726349,1702267776392927486576,1642246141129997801778847,1641699764139360487483239,887806114,54736,...,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False,2022-12-04 23:12:59
73,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19329,0,435238970363534,3000407417762830870845360,2999931610314728238726349,1460557767872656691963,1358687420629498588455174,1358333271081056393600317,797470641,45283,...,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False,2022-12-03 23:30:59
74,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19328,0,424429606966213,3000316417810236657492429,2999931610314728238726349,1298896246642712535146,1287538705363615135157340,1287292159028786848067817,819794982,42913,...,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False,2022-12-02 23:55:59
75,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19327,0,455232217741157,1000250018374254924219488,1000177511746822029696096,11654430437070525

*************************************************************************
SILO - ah turns out we haven't launched yet as of 02/11

In [ ]:
siloInterestDailySnapshotTicker = 0
siloInterestDailySnapshotslist = pd.DataFrame()
siloInterestDailySnapshotsDb = pd.DataFrame()
siloInterestDailySnapshotListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (siloInterestDailySnapshotTicker)*(1000)
    siloInterestDailySnapshots = silo.Query.interestRateDailies(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip= skipValue,
        #where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' } #gOHM market
        #where = {'day': 19400, 'timestamp':1676160000}
        where = {'interestRate_in': ['LENDER-VARIABLE-0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db-0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac','LENDER-VARIABLE-0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db-0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2','LENDER-VARIABLE-0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db-0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5','LENDER-VARIABLE-0xc413dd03555f3eb29d834b482d386b2999dc2eb0-0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac','LENDER-VARIABLE-0xc413dd03555f3eb29d834b482d386b2999dc2eb0-0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2','LENDER-VARIABLE-0xc413dd03555f3eb29d834b482d386b2999dc2eb0-0x0ab87046fbb341d058f17cbc4c1133f25a20a52f','BORROWER-VARIABLE-0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db-0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac','BORROWER-VARIABLE-0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db-0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2','BORROWER-VARIABLE-0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db-0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5','BORROWER-VARIABLE-0xc413dd03555f3eb29d834b482d386b2999dc2eb0-0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac','BORROWER-VARIABLE-0xc413dd03555f3eb29d834b482d386b2999dc2eb0-0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2']}
    )

    siloInterestDailySnapshotslist = sg.query_df([
        siloInterestDailySnapshots.interestRate.market.name
        ,siloInterestDailySnapshots.id
        ,siloInterestDailySnapshots.day
        ,siloInterestDailySnapshots.timestamp
        ,siloInterestDailySnapshots.rateLast #The most recent rate recorded or last for the day
        ,siloInterestDailySnapshots.rateHigh #Highest the rate was for the day
        ,siloInterestDailySnapshots.rateLow
        ,siloInterestDailySnapshots.rateAvg  #Interest rate in percentage APY. E.g. 5.21% should be stored as 5.21
        ,siloInterestDailySnapshots.interestRate.side
        ,siloInterestDailySnapshots.interestRate.type
    ])

    #siloInterestDailySnapshotslist.sort_values(by=['siloInterestDailySnapshots_timestamp'])

    siloInterestDailySnapshotsDb=pd.concat([siloInterestDailySnapshotsDb, siloInterestDailySnapshotslist])
    recordTimestamp1 = siloInterestDailySnapshotsDb.iat[siloInterestDailySnapshotTicker,3]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    siloInterestDailySnapshotsDbLength = len(siloInterestDailySnapshotsDb)
    siloInterestDailySnapshotListLength = len(siloInterestDailySnapshotslist)
    recordID = siloInterestDailySnapshotsDb.iat[siloInterestDailySnapshotTicker,0]
    siloInterestDailySnapshot = siloInterestDailySnapshotsDb.iat[siloInterestDailySnapshotTicker,1]
    print("iterations: ", siloInterestDailySnapshotTicker, "Lines skipped: ",skipValue, "records collected: ", siloInterestDailySnapshotListLength, " - latest recordID: ",siloInterestDailySnapshot, " ", recordID , " - siloInterestDailySnapshots DB length: ", siloInterestDailySnapshotsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", siloInterestDailySnapshotTicker, "records collected: ", siloInterestDailySnapshotListLength, " - userdb length: ", siloInterestDailySnapshotsDbLength)
    siloInterestDailySnapshotTicker = siloInterestDailySnapshotTicker+1

    if daysAgo>daysLimit: exit =True
    if siloInterestDailySnapshotListLength<1000: exit =True
print("Done. Total Records collected: ", siloInterestDailySnapshotsDbLength)

In [ ]:
print(len(siloInterestDailySnapshotsDb))
siloInterestDailySnapshotsDb.drop_duplicates
print(len(siloInterestDailySnapshotsDb))

In [ ]:
siloInterestDailySnapshotsDbTimed= db.query("select "
                   "    * "
                        " ,to_timestamp(interestRateDailies_timestamp) dateTime "
                   "From siloInterestDailySnapshotsDb "
                   #"where lower(liquidityPools_name) like lower('%olympus%') "
                    #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
                   #" limit 100 "
                   ).df()
siloInterestDailySnapshotsDbTimed.head(1000)

In [ ]:
siloInterestDailySnapshotsDbTimedLength = len(siloInterestDailySnapshotsDbTimed)
path =file+'/siloInterestDailySnapshotsDbTimedLength_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(siloInterestDailySnapshotsDbTimedLength)+'_records.csv'
siloInterestDailySnapshotsDbTimed.to_csv(path, index = False)

path =file+'/siloInterestDailySnapshotsDbTimedLength_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(siloInterestDailySnapshotsDbTimedLength)+'_records_piped.csv'
siloInterestDailySnapshotsDbTimed.to_csv(path, index = False, sep='|')
print('file saved')

In [ ]:
siloDailySnapshot = silo.Query.marketDailySnapshots(
    orderBy='timestamp',
    orderDirection='desc',
    first=10,
    #skip = skipValue,
    where={              'market':'0xc413dd03555f3eb29d834b482d386b2999dc2eb0'            } #gOHM market
    #where={              'market':'0xfccc27aabd0ab7a0b2ad2b7760037b1eab61616b'            } #DAI for testing
    #0x0ab87046fbb341d058f17cbc4c1133f25a20a52f
)

siloDailySnapshotslist = sg.query_df([
    siloDailySnapshot.protocol.name
    ,siloDailySnapshot.protocol.type
    ,siloDailySnapshot.protocol.cumulativeUniqueUsers
    ,siloDailySnapshot.protocol.cumulativeUniqueDepositors
    ,siloDailySnapshot.protocol.cumulativeUniqueBorrowers
    ,siloDailySnapshot.protocol.cumulativeUniqueLiquidators
    ,siloDailySnapshot.protocol.cumulativeUniqueLiquidatees
    ,siloDailySnapshot.protocol.totalValueLockedUSD
    ,siloDailySnapshot.protocol.protocolControlledValueUSD
    ,siloDailySnapshot.protocol.cumulativeSupplySideRevenueUSD
    ,siloDailySnapshot.protocol.cumulativeProtocolSideRevenueUSD
    ,siloDailySnapshot.protocol.cumulativeTotalRevenueUSD
    ,siloDailySnapshot.protocol.totalDepositBalanceUSD
    ,siloDailySnapshot.protocol.cumulativeDepositUSD
    ,siloDailySnapshot.protocol.cumulativeWithdrawUSD
    ,siloDailySnapshot.protocol.totalBorrowBalanceUSD
    ,siloDailySnapshot.protocol.totalBorrowBalanceUSD
    ,siloDailySnapshot.protocol.cumulativeLiquidateUSD
    ,siloDailySnapshot.protocol.creditSupply
    ,siloDailySnapshot.protocol.creditUtilized
    ,siloDailySnapshot.market.id
    ,siloDailySnapshot.market.name
    ,siloDailySnapshot.market.totalValueLockedUSD
    #,siloDailySnapshot.market.totalControlledValueUSD
    ,siloDailySnapshot.market.cumulativeSupplySideRevenueUSD
    ,siloDailySnapshot.market.cumulativeTotalRevenueUSD
    ,siloDailySnapshot.market.cumulativeSupplySideRevenueUSD
    ,siloDailySnapshot.market.cumulativeProtocolSideRevenueUSD
    ,siloDailySnapshot.market.totalDepositBalanceUSD
    ,siloDailySnapshot.market.cumulativeDepositUSD
    ,siloDailySnapshot.market.totalBorrowBalanceUSD
    ,siloDailySnapshot.market.cumulativeBorrowUSD
    ,siloDailySnapshot.market.cumulativeLiquidateUSD
    ,siloDailySnapshot.market.creditSupply
    ,siloDailySnapshot.market.creditUtilized
    ,siloDailySnapshot.timestamp
])

siloDailySnapshotslistTimed= db.query("select "
                   "    * "
                        " ,to_timestamp(marketDailySnapshots_timestamp) dateTime "
                   "From siloDailySnapshotslist "
                   #"where lower(liquidityPools_name) like lower('%olympus%') "
                    #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
                   #" limit 100 "
                   ).df()
siloDailySnapshotslistTimed.drop_duplicates
siloDailySnapshotslistTimed.head(1000)

In [ ]:
siloDailySnapshot = silo.Query.marketDailySnapshots(
    orderBy='timestamp',
    orderDirection='desc',
    #first=1000,
    #skip = skipValue,
    where={ 'id': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0-19398'
            ,'protocol':'0xd998c35b7900b344bbbe6555cc11576942cf309d'
            #,'market':'gOHM'
            } #gOHM market
    #0x0ab87046fbb341d058f17cbc4c1133f25a20a52f
)

siloDailySnapshotslist = sg.query_df([
    siloDailySnapshot.id
    ,siloDailySnapshot.protocol
    ,siloDailySnapshot.market
    ,siloDailySnapshot.timestamp
])
siloDailySnapshotslist.head(10)

*****************************************************
THIS... IS.... VESTAAAAAAAAAAA!!
(kick)

In [ ]:
##this captures the ENTIRE list of people who vestaDailySnapshoted XAI against gOHM
vestaDailySnapshotTicker = 0
vestaDailySnapshotslist = pd.DataFrame()
vestaDailySnapshotsDb = pd.DataFrame()
vestaDailySnapshotListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (vestaDailySnapshotTicker)*(1000)
    vestaDailySnapshots = vesta.Query.MarketDailySnapshot(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1' } #gOHM market

    )

    vestaDailySnapshotslist = sg.query_df([
        vestaDailySnapshots.id
        ,vestaDailySnapshots.market
        ,vestaDailySnapshots.rates
    ])

    #vestaDailySnapshotslist.sort_values(by=['vestaDailySnapshots_timestamp'])

    vestaDailySnapshotsDb=pd.concat([vestaDailySnapshotsDb, vestaDailySnapshotslist])
    recordTimestamp1 = vestaDailySnapshotsDb.iat[vestaDailySnapshotTicker,2]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    vestaDailySnapshotsDbLength = len(vestaDailySnapshotsDb)
    vestaDailySnapshotListLength = len(vestaDailySnapshotslist)
    recordID = vestaDailySnapshotsDb.iat[vestaDailySnapshotTicker,0]
    vestaDailySnapshot = vestaDailySnapshotsDb.iat[vestaDailySnapshotTicker,1]
    print("iterations: ", vestaDailySnapshotTicker, "Lines skipped: ",skipValue, "records collected: ", vestaDailySnapshotListLength, " - latest recordID: ",vestaDailySnapshot, " ", recordID , " - vestaDailySnapshots DB length: ", vestaDailySnapshotsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", vestaDailySnapshotTicker, "records collected: ", vestaDailySnapshotListLength, " - userdb length: ", vestaDailySnapshotsDbLength)
    vestaDailySnapshotTicker = vestaDailySnapshotTicker+1

    if daysAgo>daysLimit: exit =True
    if vestaDailySnapshotListLength<1000: exit =True


vestaDailySnapshotsDb.columns = vestaDailySnapshotsDb.columns.str.replace("vestaDailySnapshots", "transaction")
vestaDailySnapshotsDb['event']='vestaDailySnapshot'
print("Done. Total Records collected: ", vestaDailySnapshotsDbLength)

In [ ]:
vestaDailySnapshots = vesta.Query.marketDailySnapshots(
    orderBy='timestamp',
    orderDirection='desc',
    first=1000,
    #where={ 'market': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1' } #gOHM market
    #WHERE = {'interestRateDailies.interestRate.market.name': 'XAI'}
    #where = {'marketDailySnapshots.market.name': 'Olympus Governance Token'}
)

vestaDailySnapshotslist = sg.query_df([
    vestaDailySnapshots.id
    ,vestaDailySnapshots.timestamp
    ,vestaDailySnapshots.market
    ,vestaDailySnapshots.rates
    ,vestaDailySnapshots.totalValueLockedUSD
    ,vestaDailySnapshots.dailyBorrowUSD
    ,vestaDailySnapshots.dailyDepositUSD
    ,vestaDailySnapshots.dailyBorrowUSD
    ,vestaDailySnapshots.dailyProtocolSideRevenueUSD
    ,vestaDailySnapshots.cumulativeProtocolSideRevenueUSD




],
#pagination_strategy=ShallowStrategy
)
vestaDailySnapshotslist.head(1000)

In [ ]:
##Loan LEngth
fraxLoanLengthDB = db.query("select distinct "
                        "    A.actions_root_id "
                            ",sum(A.actions_amountUSD) totalBorrowed "
                            ",sum(B.actions_amountUSD) totalRepaid "
                            " ,sum(A.actions_amountUSD) - sum(B.actions_amountUSD) balance "
                        #" ,date_diff('day',date_trunc('day',A.dateTime), date_trunc('day',B.dateTime)) datediff "
                   "From fraxBorrowsDb A "
                            "Join fraxRepaysDb B on A.actions_root_id = B.actions_root_id"
                   #"where lower(liquidityPools_name) like lower('%olympus%') "
                    #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
                   " group by 1 "
                   " limit 100 "
                   ).df()


fraxLoanLengthDB.head(10)


In [ ]:
vestaDailySnapshot = vesta.Query.marketDailySnapshots(
    orderBy='timestamp',
    orderDirection='desc',
    first=10,
    #skip = skipValue,
    where={              'market':'0xc413dd03555f3eb29d834b482d386b2999dc2eb0'            } #gOHM market
    #where={              'market':'0xfccc27aabd0ab7a0b2ad2b7760037b1eab61616b'            } #DAI for testing
    #0x0ab87046fbb341d058f17cbc4c1133f25a20a52f
)

vestaDailySnapshotslist = sg.query_df([
    vestaDailySnapshot.protocol.name
    ,vestaDailySnapshot.protocol.type
    ,vestaDailySnapshot.protocol.cumulativeUniqueUsers
    ,vestaDailySnapshot.protocol.cumulativeUniqueDepositors
    ,vestaDailySnapshot.protocol.cumulativeUniqueBorrowers
    ,vestaDailySnapshot.protocol.cumulativeUniqueLiquidators
    ,vestaDailySnapshot.protocol.cumulativeUniqueLiquidatees
    ,vestaDailySnapshot.protocol.totalValueLockedUSD
    ,vestaDailySnapshot.protocol.protocolControlledValueUSD
    ,vestaDailySnapshot.protocol.cumulativeSupplySideRevenueUSD
    ,vestaDailySnapshot.protocol.cumulativeProtocolSideRevenueUSD
    ,vestaDailySnapshot.protocol.cumulativeTotalRevenueUSD
    ,vestaDailySnapshot.protocol.totalDepositBalanceUSD
    ,vestaDailySnapshot.protocol.cumulativeDepositUSD
    ,vestaDailySnapshot.protocol.cumulativeWithdrawUSD
    ,vestaDailySnapshot.protocol.totalBorrowBalanceUSD
    ,vestaDailySnapshot.protocol.totalBorrowBalanceUSD
    ,vestaDailySnapshot.protocol.cumulativeLiquidateUSD
    ,vestaDailySnapshot.protocol.creditSupply
    ,vestaDailySnapshot.protocol.creditUtilized
    ,vestaDailySnapshot.market.id
    ,vestaDailySnapshot.market.name
    ,vestaDailySnapshot.market.totalValueLockedUSD
    #,vestaDailySnapshot.market.totalControlledValueUSD
    ,vestaDailySnapshot.market.cumulativeSupplySideRevenueUSD
    ,vestaDailySnapshot.market.cumulativeTotalRevenueUSD
    ,vestaDailySnapshot.market.cumulativeSupplySideRevenueUSD
    ,vestaDailySnapshot.market.cumulativeProtocolSideRevenueUSD
    ,vestaDailySnapshot.market.totalDepositBalanceUSD
    ,vestaDailySnapshot.market.cumulativeDepositUSD
    ,vestaDailySnapshot.market.totalBorrowBalanceUSD
    ,vestaDailySnapshot.market.cumulativeBorrowUSD
    ,vestaDailySnapshot.market.cumulativeLiquidateUSD
    ,vestaDailySnapshot.market.creditSupply
    ,vestaDailySnapshot.market.creditUtilized
    ,vestaDailySnapshot.timestamp
])

vestaDailySnapshotslistTimed= db.query("select "
                   "    * "
                        " ,to_timestamp(marketDailySnapshots_timestamp) dateTime "
                   "From vestaDailySnapshotslist "
                   #"where lower(liquidityPools_name) like lower('%olympus%') "
                    #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
                   #" limit 100 "
                   ).df()
vestaDailySnapshotslistTimed.drop_duplicates
vestaDailySnapshotslistTimed.head(1000)

In [ ]:
#list of borrowers

borrowerList=db.query(
    "select distinct actions_root_id as wallet_address,'frax' as protocol from fraxBorrowsDb "
    "UNION ALL "
    "select distinct borrows_account_id as wallet_address , 'vesta' as protocol  from vestaBorrowsDbTimed "
    "UNION ALL "
    "select distinct transaction_account_id as wallet_address, 'silo' as protocol from borrowsDbTimed "
).df()

borrowerList.head(1000)


In [ ]:
borrowerListLength = len(borrowerList)
path =file+'/borrower_list_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(borrowerListLength)+'_records.csv'
borrowerList.to_csv(path, index = False)